### For the rosenbrock function and the model 4A function, implement the gradient descent algorithm for fixed step size and the variable step size.
### Implement table 4.1 and table 4.2 of the lecture notes for the rosenbrock function and the model 4A function.
### The homework for this we need to complete by Wednesday (31st September 2025)

# Rosenbrock function and its gradient
$m(u) = 10 \big(y - (x)^2\big)^2 + (x - 1)^2$


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def rosenbrock(x, y):
    return 10 * (y - (x ** 2)) ** 2 + (x - 1) ** 2

def gradient_rosenbrock(x, y):
    return np.array([-40*x*(-x**2 + y) + 2*x - 2,-20*x**2 + 20*y])

x, y = [-0.75, 0.7]
gradient = gradient_rosenbrock(x, y)
    
print(f"(x, y) = ({x:6.3f}, {y:6.3f}) | "
          f"rosenbrock = {rosenbrock(x, y):10.6f} | "
          f"gradient = ({gradient[0]:10.6f}, {gradient[1]:10.6f})")

(x, y) = (-0.750,  0.700) | rosenbrock =   3.251562 | gradient = (  0.625000,   2.750000)


In [5]:
def gradient_descent_rosenbrock(alpha=0.001, u_init=np.array([-0.75, 0.7]), tol=1e-6, max_iter=10000):
    """
    Implements the steepest descent algorithm for the Rosenbrock function.
    
    Parameters:
        alpha     : float   – step size (learning rate)
        u_init    : np.array – initial guess [x, y]
        tol       : float   – tolerance for stopping criterion
        max_iter  : int     – maximum number of iterations
        
    Returns:
        u_star : np.array – the found minimum point
        m_star : float    – minimum value of the function
        history: list     – list of (u, m) for each iteration
    """
    u = np.array(u_init, dtype=float)
    m_new = rosenbrock(u[0], u[1])
    m_old = 1e10  # large dummy value
    history = [(u.copy(), m_new)]
    
    i = 0
    while m_new < m_old and i < max_iter:
        m_old = m_new
        f = gradient_rosenbrock(u[0], u[1])  # ∂m/∂u
        h = -f                               # descent direction
        u = u + alpha * h                    # update rule
        m_new = rosenbrock(u[0], u[1])
        history.append((u.copy(), m_new))
        if np.linalg.norm(f) < tol:
            break
        i += 1

    u_star = u - alpha * h  # final update per pseudocode
    m_star = m_old
    return u_star, m_star, history


# Example usage:
u_star, m_star, history = gradient_descent_rosenbrock(alpha=0.001, u_init=np.array([-0.75, 0.7]))

print(f"Optimal point u* = {u_star}")
print(f"Minimum value m* = {m_star:.6f}")
print(f"Iterations = {len(history)}")

Optimal point u* = [0.99273961 0.98523907]
Minimum value m* = 0.000054
Iterations = 10001


In [7]:
def steepest_descent_armijo(
    u_init=np.array([-0.75, 0.7]),
    alpha_init=1.0,
    c=1e-4,      # smaller Armijo parameter for stability
    r=0.5,       # step reduction factor
    tol=1e-10,   # convergence tolerance (on gradient norm)
    max_iter=20000
):
    """
    Steepest Descent algorithm with Armijo backtracking line search.

    Parameters
    ----------
    u_init : np.array
        Initial guess [x, y]
    alpha_init : float
        Initial step size
    c : float
        Armijo parameter (typical range 1e-4 to 1e-1)
    r : float
        Step size reduction factor (0 < r < 1)
    tol : float
        Gradient norm tolerance for stopping
    max_iter : int
        Maximum iterations

    Returns
    -------
    u_star : np.array
        Approximate minimizer
    m_star : float
        Function value at minimizer
    iters : int
        Number of iterations used
    history : list
        List of (u, m, alpha)
    """
    u = np.array(u_init, dtype=float)
    m = rosenbrock(u[0], u[1])
    history = [(u.copy(), m, alpha_init)]

    for i in range(max_iter):
        grad = gradient_rosenbrock(u[0], u[1])
        grad_norm = np.linalg.norm(grad)
        if grad_norm < tol:
            break  # convergence achieved

        h = -grad
        alpha = alpha_init

        # Backtracking line search using Armijo rule
        while True:
            u_trial = u + alpha * h
            m_trial = rosenbrock(u_trial[0], u_trial[1])
            if m_trial <= m + c * alpha * np.dot(h, grad):
                break
            alpha *= r

        # Accept new point
        u = u_trial
        m = m_trial
        history.append((u.copy(), m, alpha))

    return u, m, i + 1, history


# Run the algorithm
u_star, m_star, iters, history = steepest_descent_armijo(
    u_init=np.array([-0.75, 0.7]),
    alpha_init=1.0,
    c=1e-4,   # recommended smaller value
    r=0.5
)

print("Optimal point (u*) =", u_star)
print("Function value (m*) =", m_star)
print("Iterations =", iters)

Optimal point (u*) = [1. 1.]
Function value (m*) = 6.42739379579334e-21
Iterations = 2444


# Model 4A and its gradient
The following expression represents the **Model 4a function**:

$$
m(u_1, u_2, \ldots, u_{80}) = - g ^ T u \;+\;
\sum_{i=1}^{10} a_i \left( \sqrt{ (u_{2i-1})^2 + (1 + u_{2i})^2 } - 1 \right)^2
$$

$$
+ \sum_{i=1}^{9} a_{i+10} \left( \sqrt{ (1 + u_{2i+1})^2 + (1 + u_{2i+2})^2 } - \sqrt{2} \right)^2
$$

$$
+ \sum_{i=1}^{9} a_{i+19} \left( \sqrt{ (1 - u_{2i-1})^2 + (1 + u_{2i})^2 } - \sqrt{2} \right)^2
$$

$$
+ \sum_{i=1}^{30} a_{i+28} \left( \sqrt{ (1 + u_{2i+20} - u_{2i})^2 + (u_{2i+19} - u_{2i-1})^2 } - 1 \right)^2
$$

$$
+ \sum_{i=1}^{36} a_{i+58} \left( \sqrt{ \big(1 + u_{2i+1+2\lfloor \tfrac{i-1}{9} \rfloor} - u_{2i-1+2\lfloor \tfrac{i-1}{9} \rfloor}\big)^2 
+ \big(u_{2i+2+2\lfloor \tfrac{i-1}{9} \rfloor} - u_{2i+2\lfloor \tfrac{i-1}{9} \rfloor}\big)^2 } - 1 \right)^2
$$

$$
+ \sum_{i=1}^{27} a_{i+94} \left( \sqrt{ \big(1 + u_{2i+21+2\lfloor \tfrac{i-1}{9} \rfloor} - u_{2i-1+2\lfloor \tfrac{i-1}{9} \rfloor}\big)^2 
+ \big(1 + u_{2i+22+2\lfloor \tfrac{i-1}{9} \rfloor} - u_{2i+2\lfloor \tfrac{i-1}{9} \rfloor}\big)^2 } - \sqrt{2} \right)^2
$$

$$
+ \sum_{i=1}^{27} a_{i+121} \left( \sqrt{ \big(1 - u_{2i+19+2\lfloor \tfrac{i-1}{9} \rfloor} + u_{2i+1+2\lfloor \tfrac{i-1}{9} \rfloor}\big)^2 
+ \big(1 + u_{2i+20+2\lfloor \tfrac{i-1}{9} \rfloor} - u_{2i+2+2\lfloor \tfrac{i-1}{9} \rfloor}\big)^2 } - \sqrt{2} \right)^2
$$


In [3]:
def model_4a(u, g, a = np.ones(80, dtype= np.int8)):
    """
        The function takes in 2 parameters namely g and u which are vectors from the 80th dimension euclidean space and calculates
        the objective function for that.
    """
     # you can write the dot multiplication this way as well g @ u
    m = -g @ u

    # Sum 1: i=1 to 10
    for i in range(1, 11):
        m += (np.sqrt(u[2 * i - 2]**2 + (1 + u[2 * i - 1]) ** 2) - 1) ** 2

    # Sum 2: i=1 to 9
    for i in range(1, 10):
        m += (np.sqrt((1 + u[2 * i]) ** 2 + (1 + u[2 * i + 1]) ** 2) - (2 ** (1 / 2))) ** 2

    # Sum 3: i=1 to 9
    for i in range(1, 10):
        m += (np.sqrt((1 - u[2 * i - 2]) ** 2 + (1 + u[2 * i - 1]) ** 2) - (2 ** (1 / 2))) ** 2

    # Sum 4: i=1 to 30
    for i in range(1, 31):
        m += (np.sqrt((1 + u[2 * i + 19] - u[2 * i - 1]) ** 2 + 
                         (u[2 * i + 18] - u[2 * i - 2]) ** 2) - 1) ** 2

    # Sum 5: i=1 to 36
    for i in range(1, 37):
        floor_val = (i - 1) // 9
        m += (np.sqrt((1 + u[2 * i + 2 *  floor_val] - u[2 * i - 2 + 2 * floor_val]) ** 2 + 
                         (u[2 * i + 1 + 2 * floor_val] - u[2 * i - 1 + 2 * floor_val]) ** 2) - 1) ** 2

    # Sum 6: i=1 to 27
    for i in range(1, 28):
        floor_val = (i - 1) // 9
        m += (np.sqrt((1 + u[2 * i + 20 + 2 * floor_val] - u[2 * i - 2 + 2 * floor_val]) ** 2 + 
                         (1 + u[2 * i + 21 + 2 * floor_val] - u[2 * i - 1 + 2 * floor_val]) ** 2) - (2 ** (1 / 2)))**2

    # Sum 7: i=1 to 27
    for i in range(1, 28):
        floor_val = (i - 1) // 9
        m += (np.sqrt((1 - u[2 * i + 18 + 2 * floor_val] + u[2 * i + 2 * floor_val]) ** 2 + 
                         (1 + u[2 * i + 19 + 2 * floor_val] - u[2 * i + 1 + 2 * floor_val]) ** 2) - (2 ** (1 / 2)))**2

    return m

def grad(g, u):

    ans = np.ones(80)

    ans[0] = (
        -g[0]
        + 2*u[0]*(np.sqrt(u[0]**2 + (u[1] + 1)**2) - 1)/np.sqrt(u[0]**2 + (u[1] + 1)**2)
        + 2*(u[0] - 1)*(np.sqrt((1 - u[0])**2 + (u[1] + 1)**2) - np.sqrt(2))/np.sqrt((1 - u[0])**2 + (u[1] + 1)**2)
        + 2*(u[0] - u[20])*(np.sqrt((-u[0] + u[20])**2 + (-u[1] + u[21] + 1)**2) - 1)/np.sqrt((-u[0] + u[20])**2 + (-u[1] + u[21] + 1)**2)
        + 2*(np.sqrt((-u[0] + u[22] + 1)**2 + (-u[1] + u[23] + 1)**2) - np.sqrt(2))*(u[0] - u[22] - 1)/np.sqrt((-u[0] + u[22] + 1)**2 + (-u[1] + u[23] + 1)**2)
        + 2*(np.sqrt((-u[1] + u[3])**2 + (-u[0] + u[2] + 1)**2) - 1)*(u[0] - u[2] - 1)/np.sqrt((-u[1] + u[3])**2 + (-u[0] + u[2] + 1)**2)
    )

    ans[1] = (
        -g[1]
        + 2*(u[1] + 1)*(np.sqrt((1 - u[0])**2 + (u[1] + 1)**2) - np.sqrt(2)) / np.sqrt((1 - u[0])**2 + (u[1] + 1)**2)
        + 2*(u[1] - u[3])*(np.sqrt((-u[1] + u[3])**2 + (-u[0] + u[2] + 1)**2) - 1) / np.sqrt((-u[1] + u[3])**2 + (-u[0] + u[2] + 1)**2)
        + 2*(np.sqrt((-u[0] + u[22] + 1)**2 + (-u[1] + u[23] + 1)**2) - np.sqrt(2))*(u[1] - u[23] - 1) / np.sqrt((-u[0] + u[22] + 1)**2 + (-u[1] + u[23] + 1)**2)
        + 2*(np.sqrt((-u[0] + u[20])**2 + (-u[1] + u[21] + 1)**2) - 1)*(u[1] - u[21] - 1) / np.sqrt((-u[0] + u[20])**2 + (-u[1] + u[21] + 1)**2)
        + 2*(u[1] + 1)*(np.sqrt(u[0]**2 + (u[1] + 1)**2) - 1) / np.sqrt(u[0]**2 + (u[1] + 1)**2)
    )

    ans[2] = (
        -g[2] 
        + 2 * u[2] * (np.sqrt(u[2]**2 + (u[3] + 1)**2) - 1) / np.sqrt(u[2]**2 + (u[3] + 1)**2)
        + 2 * (-u[22] + u[2]) * (np.sqrt((u[22] - u[2])**2 + (u[23] - u[3] + 1)**2) - 1) / np.sqrt((u[22] - u[2])**2 + (u[23] - u[3] + 1)**2)
        + 2 * (u[2] - 1) * (np.sqrt((1 - u[2])**2 + (u[3] + 1)**2) - np.sqrt(2)) / np.sqrt((1 - u[2])**2 + (u[3] + 1)**2)
        + 2 * (u[2] + 1) * (np.sqrt((u[2] + 1)**2 + (u[3] + 1)**2) - np.sqrt(2)) / np.sqrt((u[2] + 1)**2 + (u[3] + 1)**2)
        + 2 * (np.sqrt((-u[20] + u[2] + 1)**2 + (u[21] - u[3] + 1)**2) - np.sqrt(2)) * (-u[20] + u[2] + 1) / np.sqrt((-u[20] + u[2] + 1)**2 + (u[21] - u[3] + 1)**2)
        + 2 * (np.sqrt((u[24] - u[2] + 1)**2 + (u[25] - u[3] + 1)**2) - np.sqrt(2)) * (-u[24] + u[2] - 1) / np.sqrt((u[24] - u[2] + 1)**2 + (u[25] - u[3] + 1)**2)
        + 2 * (np.sqrt((-u[3] + u[5])**2 + (-u[2] + u[4] + 1)**2) - 1) * (u[2] - u[4] - 1) / np.sqrt((-u[3] + u[5])**2 + (-u[2] + u[4] + 1)**2)
        + 2 * (np.sqrt((-u[1] + u[3])**2 + (-u[0] + u[2] + 1)**2) - 1) * (-u[0] + u[2] + 1) / np.sqrt((-u[1] + u[3])**2 + (-u[0] + u[2] + 1)**2)
    )

    ans[3] = (
        -g[3]
        + 2 * (-u[1] + u[3]) * (np.sqrt((-u[1] + u[3])**2 + (-u[0] + u[2] + 1)**2) - 1) / np.sqrt((-u[1] + u[3])**2 + (-u[0] + u[2] + 1)**2)
        + 2 * (u[3] + 1) * (np.sqrt((u[2] + 1)**2 + (u[3] + 1)**2) - np.sqrt(2)) / np.sqrt((u[2] + 1)**2 + (u[3] + 1)**2)
        + 2 * (u[3] + 1) * (np.sqrt((1 - u[2])**2 + (u[3] + 1)**2) - np.sqrt(2)) / np.sqrt((1 - u[2])**2 + (u[3] + 1)**2)
        + 2 * (u[3] - u[5]) * (np.sqrt((-u[3] + u[5])**2 + (-u[2] + u[4] + 1)**2) - 1) / np.sqrt((-u[3] + u[5])**2 + (-u[2] + u[4] + 1)**2)
        + 2 * (np.sqrt((-u[20] + u[2] + 1)**2 + (u[21] - u[3] + 1)**2) - np.sqrt(2)) * (-u[21] + u[3] - 1) / np.sqrt((-u[20] + u[2] + 1)**2 + (u[21] - u[3] + 1)**2)
        + 2 * (np.sqrt((u[24] - u[2] + 1)**2 + (u[25] - u[3] + 1)**2) - np.sqrt(2)) * (-u[25] + u[3] - 1) / np.sqrt((u[24] - u[2] + 1)**2 + (u[25] - u[3] + 1)**2)
        + 2 * (np.sqrt((u[22] - u[2])**2 + (u[23] - u[3] + 1)**2) - 1) * (-u[23] + u[3] - 1) / np.sqrt((u[22] - u[2])**2 + (u[23] - u[3] + 1)**2)
        + 2 * (u[3] + 1) * (np.sqrt(u[2]**2 + (u[3] + 1)**2) - 1) / np.sqrt(u[2]**2 + (u[3] + 1)**2)
    )

    ans[4] = (
        -g[4]
        + 2 * u[4] * (np.sqrt(u[4]**2 + (u[5] + 1)**2) - 1) / np.sqrt(u[4]**2 + (u[5] + 1)**2)
        + 2 * (-u[24] + u[4]) * (np.sqrt((u[24] - u[4])**2 + (u[25] - u[5] + 1)**2) - 1) / np.sqrt((u[24] - u[4])**2 + (u[25] - u[5] + 1)**2)
        + 2 * (u[4] - 1) * (np.sqrt((1 - u[4])**2 + (u[5] + 1)**2) - np.sqrt(2)) / np.sqrt((1 - u[4])**2 + (u[5] + 1)**2)
        + 2 * (u[4] + 1) * (np.sqrt((u[4] + 1)**2 + (u[5] + 1)**2) - np.sqrt(2)) / np.sqrt((u[4] + 1)**2 + (u[5] + 1)**2)
        + 2 * (np.sqrt((-u[22] + u[4] + 1)**2 + (u[23] - u[5] + 1)**2) - np.sqrt(2)) * (-u[22] + u[4] + 1) / np.sqrt((-u[22] + u[4] + 1)**2 + (u[23] - u[5] + 1)**2)
        + 2 * (np.sqrt((u[26] - u[4] + 1)**2 + (u[27] - u[5] + 1)**2) - np.sqrt(2)) * (-u[26] + u[4] - 1) / np.sqrt((u[26] - u[4] + 1)**2 + (u[27] - u[5] + 1)**2)
        + 2 * (np.sqrt((-u[5] + u[7])**2 + (-u[4] + u[6] + 1)**2) - 1) * (u[4] - u[6] - 1) / np.sqrt((-u[5] + u[7])**2 + (-u[4] + u[6] + 1)**2)
        + 2 * (np.sqrt((-u[3] + u[5])**2 + (-u[2] + u[4] + 1)**2) - 1) * (-u[2] + u[4] + 1) / np.sqrt((-u[3] + u[5])**2 + (-u[2] + u[4] + 1)**2)
    )

    ans[5] = (
        -g[5]
        + 2 * (-u[3] + u[5]) * (np.sqrt((-u[3] + u[5])**2 + (-u[2] + u[4] + 1)**2) - 1) / np.sqrt((-u[3] + u[5])**2 + (-u[2] + u[4] + 1)**2)
        + 2 * (u[5] + 1) * (np.sqrt((u[4] + 1)**2 + (u[5] + 1)**2) - np.sqrt(2)) / np.sqrt((u[4] + 1)**2 + (u[5] + 1)**2)
        + 2 * (u[5] + 1) * (np.sqrt((1 - u[4])**2 + (u[5] + 1)**2) - np.sqrt(2)) / np.sqrt((1 - u[4])**2 + (u[5] + 1)**2)
        + 2 * (u[5] - u[7]) * (np.sqrt((-u[5] + u[7])**2 + (-u[4] + u[6] + 1)**2) - 1) / np.sqrt((-u[5] + u[7])**2 + (-u[4] + u[6] + 1)**2)
        + 2 * (np.sqrt((-u[22] + u[4] + 1)**2 + (u[23] - u[5] + 1)**2) - np.sqrt(2)) * (-u[23] + u[5] - 1) / np.sqrt((-u[22] + u[4] + 1)**2 + (u[23] - u[5] + 1)**2)
        + 2 * (np.sqrt((u[26] - u[4] + 1)**2 + (u[27] - u[5] + 1)**2) - np.sqrt(2)) * (-u[27] + u[5] - 1) / np.sqrt((u[26] - u[4] + 1)**2 + (u[27] - u[5] + 1)**2)
        + 2 * (np.sqrt((u[24] - u[4])**2 + (u[25] - u[5] + 1)**2) - 1) * (-u[25] + u[5] - 1) / np.sqrt((u[24] - u[4])**2 + (u[25] - u[5] + 1)**2)
        + 2 * (u[5] + 1) * (np.sqrt(u[4]**2 + (u[5] + 1)**2) - 1) / np.sqrt(u[4]**2 + (u[5] + 1)**2)
    )

    ans[6] = (
        -g[6]
        + 2 * u[6] * (np.sqrt(u[6]**2 + (u[7] + 1)**2) - 1) / np.sqrt(u[6]**2 + (u[7] + 1)**2)
        + 2 * (-u[26] + u[6]) * (np.sqrt((u[26] - u[6])**2 + (u[27] - u[7] + 1)**2) - 1) / np.sqrt((u[26] - u[6])**2 + (u[27] - u[7] + 1)**2)
        + 2 * (u[6] - 1) * (np.sqrt((1 - u[6])**2 + (u[7] + 1)**2) - np.sqrt(2)) / np.sqrt((1 - u[6])**2 + (u[7] + 1)**2)
        + 2 * (u[6] + 1) * (np.sqrt((u[6] + 1)**2 + (u[7] + 1)**2) - np.sqrt(2)) / np.sqrt((u[6] + 1)**2 + (u[7] + 1)**2)
        + 2 * (np.sqrt((-u[24] + u[6] + 1)**2 + (u[25] - u[7] + 1)**2) - np.sqrt(2)) * (-u[24] + u[6] + 1) / np.sqrt((-u[24] + u[6] + 1)**2 + (u[25] - u[7] + 1)**2)
        + 2 * (np.sqrt((u[28] - u[6] + 1)**2 + (u[29] - u[7] + 1)**2) - np.sqrt(2)) * (-u[28] + u[6] - 1) / np.sqrt((u[28] - u[6] + 1)**2 + (u[29] - u[7] + 1)**2)
        + 2 * (np.sqrt((-u[5] + u[7])**2 + (-u[4] + u[6] + 1)**2) - 1) * (-u[4] + u[6] + 1) / np.sqrt((-u[5] + u[7])**2 + (-u[4] + u[6] + 1)**2)
        + 2 * (np.sqrt((u[9] - u[7])**2 + (-u[6] + u[8] + 1)**2) - 1) * (u[6] - u[8] - 1) / np.sqrt((u[9] - u[7])**2 + (-u[6] + u[8] + 1)**2)
    )

    ans[7] = (
        -g[7]
        + 2 * (-u[9] + u[7]) * (np.sqrt((u[9] - u[7])**2 + (-u[6] + u[8] + 1)**2) - 1) / np.sqrt((u[9] - u[7])**2 + (-u[6] + u[8] + 1)**2)
        + 2 * (-u[5] + u[7]) * (np.sqrt((-u[5] + u[7])**2 + (-u[4] + u[6] + 1)**2) - 1) / np.sqrt((-u[5] + u[7])**2 + (-u[4] + u[6] + 1)**2)
        + 2 * (u[7] + 1) * (np.sqrt((u[6] + 1)**2 + (u[7] + 1)**2) - np.sqrt(2)) / np.sqrt((u[6] + 1)**2 + (u[7] + 1)**2)
        + 2 * (u[7] + 1) * (np.sqrt((1 - u[6])**2 + (u[7] + 1)**2) - np.sqrt(2)) / np.sqrt((1 - u[6])**2 + (u[7] + 1)**2)
        + 2 * (np.sqrt((-u[24] + u[6] + 1)**2 + (u[25] - u[7] + 1)**2) - np.sqrt(2)) * (-u[25] + u[7] - 1) / np.sqrt((-u[24] + u[6] + 1)**2 + (u[25] - u[7] + 1)**2)
        + 2 * (np.sqrt((u[28] - u[6] + 1)**2 + (u[29] - u[7] + 1)**2) - np.sqrt(2)) * (-u[29] + u[7] - 1) / np.sqrt((u[28] - u[6] + 1)**2 + (u[29] - u[7] + 1)**2)
        + 2 * (np.sqrt((u[26] - u[6])**2 + (u[27] - u[7] + 1)**2) - 1) * (-u[27] + u[7] - 1) / np.sqrt((u[26] - u[6])**2 + (u[27] - u[7] + 1)**2)
        + 2 * (u[7] + 1) * (np.sqrt(u[6]**2 + (u[7] + 1)**2) - 1) / np.sqrt(u[6]**2 + (u[7] + 1)**2)
    )

    ans[8] = (
        -g[8]
        + 2 * u[8] * (np.sqrt(u[8]**2 + (u[9] + 1)**2) - 1) / np.sqrt(u[8]**2 + (u[9] + 1)**2)
        + 2 * (-u[28] + u[8]) * (np.sqrt((u[28] - u[8])**2 + (-u[9] + u[29] + 1)**2) - 1) / np.sqrt((u[28] - u[8])**2 + (-u[9] + u[29] + 1)**2)
        + 2 * (u[8] - 1) * (np.sqrt((1 - u[8])**2 + (u[9] + 1)**2) - np.sqrt(2)) / np.sqrt((1 - u[8])**2 + (u[9] + 1)**2)
        + 2 * (u[8] + 1) * (np.sqrt((u[9] + 1)**2 + (u[8] + 1)**2) - np.sqrt(2)) / np.sqrt((u[9] + 1)**2 + (u[8] + 1)**2)
        + 2 * (np.sqrt((-u[9] + u[27] + 1)**2 + (-u[26] + u[8] + 1)**2) - np.sqrt(2)) * (-u[26] + u[8] + 1) / np.sqrt((-u[9] + u[27] + 1)**2 + (-u[26] + u[8] + 1)**2)
        + 2 * (np.sqrt((-u[9] + u[31] + 1)**2 + (u[30] - u[8] + 1)**2) - np.sqrt(2)) * (-u[30] + u[8] - 1) / np.sqrt((-u[9] + u[31] + 1)**2 + (u[30] - u[8] + 1)**2)
        + 2 * (np.sqrt((u[9] - u[7])**2 + (-u[6] + u[8] + 1)**2) - 1) * (-u[6] + u[8] + 1) / np.sqrt((u[9] - u[7])**2 + (-u[6] + u[8] + 1)**2)
        + 2 * (np.sqrt((-u[9] + u[11])**2 + (u[10] - u[8] + 1)**2) - 1) * (-u[10] + u[8] - 1) / np.sqrt((-u[9] + u[11])**2 + (u[10] - u[8] + 1)**2)
    )

    ans[9] = (
        -g[9]
        + 2 * (u[9] + 1) * (np.sqrt((u[9] + 1)**2 + (u[8] + 1)**2) - np.sqrt(2)) / np.sqrt((u[9] + 1)**2 + (u[8] + 1)**2)
        + 2 * (u[9] + 1) * (np.sqrt((1 - u[8])**2 + (u[9] + 1)**2) - np.sqrt(2)) / np.sqrt((1 - u[8])**2 + (u[9] + 1)**2)
        + 2 * (u[9] + 1) * (np.sqrt(u[8]**2 + (u[9] + 1)**2) - 1) / np.sqrt(u[8]**2 + (u[9] + 1)**2)
        + 2 * (u[9] - u[11]) * (np.sqrt((-u[9] + u[11])**2 + (u[10] - u[8] + 1)**2) - 1) / np.sqrt((-u[9] + u[11])**2 + (u[10] - u[8] + 1)**2)
        + 2 * (u[9] - u[7]) * (np.sqrt((u[9] - u[7])**2 + (-u[6] + u[8] + 1)**2) - 1) / np.sqrt((u[9] - u[7])**2 + (-u[6] + u[8] + 1)**2)
        + 2 * (np.sqrt((-u[9] + u[27] + 1)**2 + (-u[26] + u[8] + 1)**2) - np.sqrt(2)) * (u[9] - u[27] - 1) / np.sqrt((-u[9] + u[27] + 1)**2 + (-u[26] + u[8] + 1)**2)
        + 2 * (np.sqrt((-u[9] + u[31] + 1)**2 + (u[30] - u[8] + 1)**2) - np.sqrt(2)) * (u[9] - u[31] - 1) / np.sqrt((-u[9] + u[31] + 1)**2 + (u[30] - u[8] + 1)**2)
        + 2 * (np.sqrt((u[28] - u[8])**2 + (-u[9] + u[29] + 1)**2) - 1) * (u[9] - u[29] - 1) / np.sqrt((u[28] - u[8])**2 + (-u[9] + u[29] + 1)**2)
    )
    ans[10] = (
        -g[10]
        + 2 * u[10] * (np.sqrt(u[10]**2 + (u[11] + 1)**2) - 1) / np.sqrt(u[10]**2 + (u[11] + 1)**2)
        + 2 * (u[10] - 1) * (np.sqrt((1 - u[10])**2 + (u[11] + 1)**2) - np.sqrt(2)) / np.sqrt((1 - u[10])**2 + (u[11] + 1)**2)
        + 2 * (u[10] + 1) * (np.sqrt((u[10] + 1)**2 + (u[11] + 1)**2) - np.sqrt(2)) / np.sqrt((u[10] + 1)**2 + (u[11] + 1)**2)
        + 2 * (u[10] - u[30]) * (np.sqrt((-u[10] + u[30])**2 + (-u[11] + u[31] + 1)**2) - 1) / np.sqrt((-u[10] + u[30])**2 + (-u[11] + u[31] + 1)**2)
        + 2 * (np.sqrt((-u[10] + u[32] + 1)**2 + (-u[11] + u[33] + 1)**2) - np.sqrt(2)) * (u[10] - u[32] - 1) / np.sqrt((-u[10] + u[32] + 1)**2 + (-u[11] + u[33] + 1)**2)
        + 2 * (np.sqrt((u[10] - u[28] + 1)**2 + (-u[11] + u[29] + 1)**2) - np.sqrt(2)) * (u[10] - u[28] + 1) / np.sqrt((u[10] - u[28] + 1)**2 + (-u[11] + u[29] + 1)**2)
        + 2 * (np.sqrt((-u[11] + u[13])**2 + (-u[10] + u[12] + 1)**2) - 1) * (u[10] - u[12] - 1) / np.sqrt((-u[11] + u[13])**2 + (-u[10] + u[12] + 1)**2)
        + 2 * (np.sqrt((-u[9] + u[11])**2 + (u[10] - u[8] + 1)**2) - 1) * (u[10] - u[8] + 1) / np.sqrt((-u[9] + u[11])**2 + (u[10] - u[8] + 1)**2)
    )

    ans[11] = (
        -g[11]
        + 2 * (-u[9] + u[11]) * (np.sqrt((-u[9] + u[11])**2 + (u[10] - u[8] + 1)**2) - 1) / np.sqrt((-u[9] + u[11])**2 + (u[10] - u[8] + 1)**2)
        + 2 * (u[11] + 1) * (np.sqrt((u[10] + 1)**2 + (u[11] + 1)**2) - np.sqrt(2)) / np.sqrt((u[10] + 1)**2 + (u[11] + 1)**2)
        + 2 * (u[11] + 1) * (np.sqrt((1 - u[10])**2 + (u[11] + 1)**2) - np.sqrt(2)) / np.sqrt((1 - u[10])**2 + (u[11] + 1)**2)
        + 2 * (u[11] - u[13]) * (np.sqrt((-u[11] + u[13])**2 + (-u[10] + u[12] + 1)**2) - 1) / np.sqrt((-u[11] + u[13])**2 + (-u[10] + u[12] + 1)**2)
        + 2 * (np.sqrt((-u[10] + u[32] + 1)**2 + (-u[11] + u[33] + 1)**2) - np.sqrt(2)) * (u[11] - u[33] - 1) / np.sqrt((-u[10] + u[32] + 1)**2 + (-u[11] + u[33] + 1)**2)
        + 2 * (np.sqrt((u[10] - u[28] + 1)**2 + (-u[11] + u[29] + 1)**2) - np.sqrt(2)) * (u[11] - u[29] - 1) / np.sqrt((u[10] - u[28] + 1)**2 + (-u[11] + u[29] + 1)**2)
        + 2 * (np.sqrt((-u[10] + u[30])**2 + (-u[11] + u[31] + 1)**2) - 1) * (u[11] - u[31] - 1) / np.sqrt((-u[10] + u[30])**2 + (-u[11] + u[31] + 1)**2)
        + 2 * (u[11] + 1) * (np.sqrt(u[10]**2 + (u[11] + 1)**2) - 1) / np.sqrt(u[10]**2 + (u[11] + 1)**2)
    )

    ans[12] = (
        -g[12]
        + 2 * u[12] * (np.sqrt(u[12]**2 + (u[13] + 1)**2) - 1) / np.sqrt(u[12]**2 + (u[13] + 1)**2)
        + 2 * (u[12] - 1) * (np.sqrt((1 - u[12])**2 + (u[13] + 1)**2) - np.sqrt(2)) / np.sqrt((1 - u[12])**2 + (u[13] + 1)**2)
        + 2 * (u[12] + 1) * (np.sqrt((u[12] + 1)**2 + (u[13] + 1)**2) - np.sqrt(2)) / np.sqrt((u[12] + 1)**2 + (u[13] + 1)**2)
        + 2 * (u[12] - u[32]) * (np.sqrt((-u[12] + u[32])**2 + (-u[13] + u[33] + 1)**2) - 1) / np.sqrt((-u[12] + u[32])**2 + (-u[13] + u[33] + 1)**2)
        + 2 * (np.sqrt((-u[12] + u[34] + 1)**2 + (-u[13] + u[35] + 1)**2) - np.sqrt(2)) * (u[12] - u[34] - 1) / np.sqrt((-u[12] + u[34] + 1)**2 + (-u[13] + u[35] + 1)**2)
        + 2 * (np.sqrt((u[12] - u[30] + 1)**2 + (-u[13] + u[31] + 1)**2) - np.sqrt(2)) * (u[12] - u[30] + 1) / np.sqrt((u[12] - u[30] + 1)**2 + (-u[13] + u[31] + 1)**2)
        + 2 * (np.sqrt((-u[13] + u[15])**2 + (-u[12] + u[14] + 1)**2) - 1) * (u[12] - u[14] - 1) / np.sqrt((-u[13] + u[15])**2 + (-u[12] + u[14] + 1)**2)
        + 2 * (np.sqrt((-u[11] + u[13])**2 + (-u[10] + u[12] + 1)**2) - 1) * (-u[10] + u[12] + 1) / np.sqrt((-u[11] + u[13])**2 + (-u[10] + u[12] + 1)**2)
    )

    ans[13] = (
        -g[13]
        + 2 * (-u[11] + u[13]) * (np.sqrt((-u[11] + u[13])**2 + (-u[10] + u[12] + 1)**2) - 1) / np.sqrt((-u[11] + u[13])**2 + (-u[10] + u[12] + 1)**2)
        + 2 * (u[13] + 1) * (np.sqrt((u[12] + 1)**2 + (u[13] + 1)**2) - np.sqrt(2)) / np.sqrt((u[12] + 1)**2 + (u[13] + 1)**2)
        + 2 * (u[13] + 1) * (np.sqrt((1 - u[12])**2 + (u[13] + 1)**2) - np.sqrt(2)) / np.sqrt((1 - u[12])**2 + (u[13] + 1)**2)
        + 2 * (u[13] - u[15]) * (np.sqrt((-u[13] + u[15])**2 + (-u[12] + u[14] + 1)**2) - 1) / np.sqrt((-u[13] + u[15])**2 + (-u[12] + u[14] + 1)**2)
        + 2 * (np.sqrt((-u[12] + u[34] + 1)**2 + (-u[13] + u[35] + 1)**2) - np.sqrt(2)) * (u[13] - u[35] - 1) / np.sqrt((-u[12] + u[34] + 1)**2 + (-u[13] + u[35] + 1)**2)
        + 2 * (np.sqrt((u[12] - u[30] + 1)**2 + (-u[13] + u[31] + 1)**2) - np.sqrt(2)) * (u[13] - u[31] - 1) / np.sqrt((u[12] - u[30] + 1)**2 + (-u[13] + u[31] + 1)**2)
        + 2 * (np.sqrt((-u[12] + u[32])**2 + (-u[13] + u[33] + 1)**2) - 1) * (u[13] - u[33] - 1) / np.sqrt((-u[12] + u[32])**2 + (-u[13] + u[33] + 1)**2)
        + 2 * (u[13] + 1) * (np.sqrt(u[12]**2 + (u[13] + 1)**2) - 1) / np.sqrt(u[12]**2 + (u[13] + 1)**2)
    )

    ans[14] = (
        -g[14]
        + 2 * u[14] * (np.sqrt(u[14]**2 + (u[15] + 1)**2) - 1) / np.sqrt(u[14]**2 + (u[15] + 1)**2)
        + 2 * (u[14] - 1) * (np.sqrt((1 - u[14])**2 + (u[15] + 1)**2) - np.sqrt(2)) / np.sqrt((1 - u[14])**2 + (u[15] + 1)**2)
        + 2 * (u[14] + 1) * (np.sqrt((u[14] + 1)**2 + (u[15] + 1)**2) - np.sqrt(2)) / np.sqrt((u[14] + 1)**2 + (u[15] + 1)**2)
        + 2 * (u[14] - u[34]) * (np.sqrt((-u[14] + u[34])**2 + (-u[15] + u[35] + 1)**2) - 1) / np.sqrt((-u[14] + u[34])**2 + (-u[15] + u[35] + 1)**2)
        + 2 * (np.sqrt((-u[14] + u[36] + 1)**2 + (-u[15] + u[37] + 1)**2) - np.sqrt(2)) * (u[14] - u[36] - 1) / np.sqrt((-u[14] + u[36] + 1)**2 + (-u[15] + u[37] + 1)**2)
        + 2 * (np.sqrt((u[14] - u[32] + 1)**2 + (-u[15] + u[33] + 1)**2) - np.sqrt(2)) * (u[14] - u[32] + 1) / np.sqrt((u[14] - u[32] + 1)**2 + (-u[15] + u[33] + 1)**2)
        + 2 * (np.sqrt((-u[15] + u[17])**2 + (-u[14] + u[16] + 1)**2) - 1) * (u[14] - u[16] - 1) / np.sqrt((-u[15] + u[17])**2 + (-u[14] + u[16] + 1)**2)
        + 2 * (np.sqrt((-u[13] + u[15])**2 + (-u[12] + u[14] + 1)**2) - 1) * (-u[12] + u[14] + 1) / np.sqrt((-u[13] + u[15])**2 + (-u[12] + u[14] + 1)**2)
    )

    ans[15] = (
        -g[15]
        + 2 * (-u[13] + u[15]) * (np.sqrt((-u[13] + u[15])**2 + (-u[12] + u[14] + 1)**2) - 1) / np.sqrt((-u[13] + u[15])**2 + (-u[12] + u[14] + 1)**2)
        + 2 * (u[15] + 1) * (np.sqrt((u[14] + 1)**2 + (u[15] + 1)**2) - np.sqrt(2)) / np.sqrt((u[14] + 1)**2 + (u[15] + 1)**2)
        + 2 * (u[15] + 1) * (np.sqrt((1 - u[14])**2 + (u[15] + 1)**2) - np.sqrt(2)) / np.sqrt((1 - u[14])**2 + (u[15] + 1)**2)
        + 2 * (u[15] - u[17]) * (np.sqrt((-u[15] + u[17])**2 + (-u[14] + u[16] + 1)**2) - 1) / np.sqrt((-u[15] + u[17])**2 + (-u[14] + u[16] + 1)**2)
        + 2 * (np.sqrt((-u[14] + u[36] + 1)**2 + (-u[15] + u[37] + 1)**2) - np.sqrt(2)) * (u[15] - u[37] - 1) / np.sqrt((-u[14] + u[36] + 1)**2 + (-u[15] + u[37] + 1)**2)
        + 2 * (np.sqrt((u[14] - u[32] + 1)**2 + (-u[15] + u[33] + 1)**2) - np.sqrt(2)) * (u[15] - u[33] - 1) / np.sqrt((u[14] - u[32] + 1)**2 + (-u[15] + u[33] + 1)**2)
        + 2 * (np.sqrt((-u[14] + u[34])**2 + (-u[15] + u[35] + 1)**2) - 1) * (u[15] - u[35] - 1) / np.sqrt((-u[14] + u[34])**2 + (-u[15] + u[35] + 1)**2)
        + 2 * (u[15] + 1) * (np.sqrt(u[14]**2 + (u[15] + 1)**2) - 1) / np.sqrt(u[14]**2 + (u[15] + 1)**2)
    )

    ans[16] = (
        -g[16]
        + 2 * u[16] * (np.sqrt(u[16]**2 + (u[17] + 1)**2) - 1) / np.sqrt(u[16]**2 + (u[17] + 1)**2)
        + 2 * (u[16] - 1) * (np.sqrt((1 - u[16])**2 + (u[17] + 1)**2) - np.sqrt(2)) / np.sqrt((1 - u[16])**2 + (u[17] + 1)**2)
        + 2 * (u[16] + 1) * (np.sqrt((u[16] + 1)**2 + (u[17] + 1)**2) - np.sqrt(2)) / np.sqrt((u[16] + 1)**2 + (u[17] + 1)**2)
        + 2 * (u[16] - u[36]) * (np.sqrt((-u[16] + u[36])**2 + (-u[17] + u[37] + 1)**2) - 1) / np.sqrt((-u[16] + u[36])**2 + (-u[17] + u[37] + 1)**2)
        + 2 * (np.sqrt((-u[16] + u[38] + 1)**2 + (-u[17] + u[39] + 1)**2) - np.sqrt(2)) * (u[16] - u[38] - 1) / np.sqrt((-u[16] + u[38] + 1)**2 + (-u[17] + u[39] + 1)**2)
        + 2 * (np.sqrt((u[16] - u[34] + 1)**2 + (-u[17] + u[35] + 1)**2) - np.sqrt(2)) * (u[16] - u[34] + 1) / np.sqrt((u[16] - u[34] + 1)**2 + (-u[17] + u[35] + 1)**2)
        + 2 * (np.sqrt((-u[17] + u[19])**2 + (-u[16] + u[18] + 1)**2) - 1) * (u[16] - u[18] - 1) / np.sqrt((-u[17] + u[19])**2 + (-u[16] + u[18] + 1)**2)
        + 2 * (np.sqrt((-u[15] + u[17])**2 + (-u[14] + u[16] + 1)**2) - 1) * (-u[14] + u[16] + 1) / np.sqrt((-u[15] + u[17])**2 + (-u[14] + u[16] + 1)**2)
    )

    ans[17] = (
        -g[17]
        + 2 * (-u[15] + u[17]) * (np.sqrt((-u[15] + u[17])**2 + (-u[14] + u[16] + 1)**2) - 1) / np.sqrt((-u[15] + u[17])**2 + (-u[14] + u[16] + 1)**2)
        + 2 * (u[17] + 1) * (np.sqrt((u[16] + 1)**2 + (u[17] + 1)**2) - np.sqrt(2)) / np.sqrt((u[16] + 1)**2 + (u[17] + 1)**2)
        + 2 * (u[17] + 1) * (np.sqrt((1 - u[16])**2 + (u[17] + 1)**2) - np.sqrt(2)) / np.sqrt((1 - u[16])**2 + (u[17] + 1)**2)
        + 2 * (u[17] - u[19]) * (np.sqrt((-u[17] + u[19])**2 + (-u[16] + u[18] + 1)**2) - 1) / np.sqrt((-u[17] + u[19])**2 + (-u[16] + u[18] + 1)**2)
        + 2 * (np.sqrt((-u[16] + u[38] + 1)**2 + (-u[17] + u[39] + 1)**2) - np.sqrt(2)) * (u[17] - u[39] - 1) / np.sqrt((-u[16] + u[38] + 1)**2 + (-u[17] + u[39] + 1)**2)
        + 2 * (np.sqrt((u[16] - u[34] + 1)**2 + (-u[17] + u[35] + 1)**2) - np.sqrt(2)) * (u[17] - u[35] - 1) / np.sqrt((u[16] - u[34] + 1)**2 + (-u[17] + u[35] + 1)**2)
        + 2 * (np.sqrt((-u[16] + u[36])**2 + (-u[17] + u[37] + 1)**2) - 1) * (u[17] - u[37] - 1) / np.sqrt((-u[16] + u[36])**2 + (-u[17] + u[37] + 1)**2)
        + 2 * (u[17] + 1) * (np.sqrt(u[16]**2 + (u[17] + 1)**2) - 1) / np.sqrt(u[16]**2 + (u[17] + 1)**2)
    )

    ans[18] = (
        -g[18]
        + 2 * u[18] * (np.sqrt(u[18]**2 + (u[19] + 1)**2) - 1) / np.sqrt(u[18]**2 + (u[19] + 1)**2)
        + 2 * (u[18] + 1) * (np.sqrt((u[18] + 1)**2 + (u[19] + 1)**2) - np.sqrt(2)) / np.sqrt((u[18] + 1)**2 + (u[19] + 1)**2)
        + 2 * (u[18] - u[38]) * (np.sqrt((-u[18] + u[38])**2 + (-u[19] + u[39] + 1)**2) - 1) / np.sqrt((-u[18] + u[38])**2 + (-u[19] + u[39] + 1)**2)
        + 2 * (np.sqrt((u[18] - u[36] + 1)**2 + (-u[19] + u[37] + 1)**2) - np.sqrt(2)) * (u[18] - u[36] + 1) / np.sqrt((u[18] - u[36] + 1)**2 + (-u[19] + u[37] + 1)**2)
        + 2 * (np.sqrt((-u[17] + u[19])**2 + (-u[16] + u[18] + 1)**2) - 1) * (-u[16] + u[18] + 1) / np.sqrt((-u[17] + u[19])**2 + (-u[16] + u[18] + 1)**2)
    )

    ans[19] = (
        -g[19]
        + 2 * (-u[17] + u[19]) * (np.sqrt((-u[17] + u[19])**2 + (-u[16] + u[18] + 1)**2) - 1) / np.sqrt((-u[17] + u[19])**2 + (-u[16] + u[18] + 1)**2)
        + 2 * (u[19] + 1) * (np.sqrt((u[18] + 1)**2 + (u[19] + 1)**2) - np.sqrt(2)) / np.sqrt((u[18] + 1)**2 + (u[19] + 1)**2)
        + 2 * (np.sqrt((u[18] - u[36] + 1)**2 + (-u[19] + u[37] + 1)**2) - np.sqrt(2)) * (u[19] - u[37] - 1) / np.sqrt((u[18] - u[36] + 1)**2 + (-u[19] + u[37] + 1)**2)
        + 2 * (np.sqrt((-u[18] + u[38])**2 + (-u[19] + u[39] + 1)**2) - 1) * (u[19] - u[39] - 1) / np.sqrt((-u[18] + u[38])**2 + (-u[19] + u[39] + 1)**2)
        + 2 * (u[19] + 1) * (np.sqrt(u[18]**2 + (u[19] + 1)**2) - 1) / np.sqrt(u[18]**2 + (u[19] + 1)**2)
    )

    ans[20] = (
        -g[20]
        + 2 * (-u[0] + u[20]) * (np.sqrt((-u[0] + u[20])**2 + (-u[1] + u[21] + 1)**2) - 1) / np.sqrt((-u[0] + u[20])**2 + (-u[1] + u[21] + 1)**2)
        + 2 * (u[20] - u[40]) * (np.sqrt((-u[20] + u[40])**2 + (-u[21] + u[41] + 1)**2) - 1) / np.sqrt((-u[20] + u[40])**2 + (-u[21] + u[41] + 1)**2)
        + 2 * (np.sqrt((-u[20] + u[2] + 1)**2 + (u[21] - u[3] + 1)**2) - np.sqrt(2)) * (u[20] - u[2] - 1) / np.sqrt((-u[20] + u[2] + 1)**2 + (u[21] - u[3] + 1)**2)
        + 2 * (np.sqrt((-u[20] + u[42] + 1)**2 + (-u[21] + u[43] + 1)**2) - np.sqrt(2)) * (u[20] - u[42] - 1) / np.sqrt((-u[20] + u[42] + 1)**2 + (-u[21] + u[43] + 1)**2)
        + 2 * (np.sqrt((-u[21] + u[23])**2 + (-u[20] + u[22] + 1)**2) - 1) * (u[20] - u[22] - 1) / np.sqrt((-u[21] + u[23])**2 + (-u[20] + u[22] + 1)**2)
    )

    ans[21] = (
        -g[21]
        + 2 * (u[21] - u[23]) * (np.sqrt((-u[21] + u[23])**2 + (-u[20] + u[22] + 1)**2) - 1) / np.sqrt((-u[21] + u[23])**2 + (-u[20] + u[22] + 1)**2)
        + 2 * (np.sqrt((-u[20] + u[2] + 1)**2 + (u[21] - u[3] + 1)**2) - np.sqrt(2)) * (u[21] - u[3] + 1) / np.sqrt((-u[20] + u[2] + 1)**2 + (u[21] - u[3] + 1)**2)
        + 2 * (np.sqrt((-u[20] + u[42] + 1)**2 + (-u[21] + u[43] + 1)**2) - np.sqrt(2)) * (u[21] - u[43] - 1) / np.sqrt((-u[20] + u[42] + 1)**2 + (-u[21] + u[43] + 1)**2)
        + 2 * (np.sqrt((-u[20] + u[40])**2 + (-u[21] + u[41] + 1)**2) - 1) * (u[21] - u[41] - 1) / np.sqrt((-u[20] + u[40])**2 + (-u[21] + u[41] + 1)**2)
        + 2 * (np.sqrt((-u[0] + u[20])**2 + (-u[1] + u[21] + 1)**2) - 1) * (-u[1] + u[21] + 1) / np.sqrt((-u[0] + u[20])**2 + (-u[1] + u[21] + 1)**2)
    )

    ans[22] = (
        -g[22]
        + 2 * (u[22] - u[2]) * (np.sqrt((u[22] - u[2])**2 + (u[23] - u[3] + 1)**2) - 1) / np.sqrt((u[22] - u[2])**2 + (u[23] - u[3] + 1)**2)
        + 2 * (u[22] - u[42]) * (np.sqrt((-u[22] + u[42])**2 + (-u[23] + u[43] + 1)**2) - 1) / np.sqrt((-u[22] + u[42])**2 + (-u[23] + u[43] + 1)**2)
        + 2 * (np.sqrt((-u[0] + u[22] + 1)**2 + (-u[1] + u[23] + 1)**2) - np.sqrt(2)) * (-u[0] + u[22] + 1) / np.sqrt((-u[0] + u[22] + 1)**2 + (-u[1] + u[23] + 1)**2)
        + 2 * (np.sqrt((-u[22] + u[44] + 1)**2 + (-u[23] + u[45] + 1)**2) - np.sqrt(2)) * (u[22] - u[44] - 1) / np.sqrt((-u[22] + u[44] + 1)**2 + (-u[23] + u[45] + 1)**2)
        + 2 * (np.sqrt((-u[22] + u[4] + 1)**2 + (u[23] - u[5] + 1)**2) - np.sqrt(2)) * (u[22] - u[4] - 1) / np.sqrt((-u[22] + u[4] + 1)**2 + (u[23] - u[5] + 1)**2)
        + 2 * (np.sqrt((u[22] - u[40] + 1)**2 + (-u[23] + u[41] + 1)**2) - np.sqrt(2)) * (u[22] - u[40] + 1) / np.sqrt((u[22] - u[40] + 1)**2 + (-u[23] + u[41] + 1)**2)
        + 2 * (np.sqrt((-u[23] + u[25])**2 + (-u[22] + u[24] + 1)**2) - 1) * (u[22] - u[24] - 1) / np.sqrt((-u[23] + u[25])**2 + (-u[22] + u[24] + 1)**2)
        + 2 * (np.sqrt((-u[21] + u[23])**2 + (-u[20] + u[22] + 1)**2) - 1) * (-u[20] + u[22] + 1) / np.sqrt((-u[21] + u[23])**2 + (-u[20] + u[22] + 1)**2)
    )

    ans[23] = (
        -g[23]
        + 2 * (-u[21] + u[23]) * (np.sqrt((-u[21] + u[23])**2 + (-u[20] + u[22] + 1)**2) - 1) / np.sqrt((-u[21] + u[23])**2 + (-u[20] + u[22] + 1)**2)
        + 2 * (u[23] - u[25]) * (np.sqrt((-u[23] + u[25])**2 + (-u[22] + u[24] + 1)**2) - 1) / np.sqrt((-u[23] + u[25])**2 + (-u[22] + u[24] + 1)**2)
        + 2 * (np.sqrt((-u[0] + u[22] + 1)**2 + (-u[1] + u[23] + 1)**2) - np.sqrt(2)) * (-u[1] + u[23] + 1) / np.sqrt((-u[0] + u[22] + 1)**2 + (-u[1] + u[23] + 1)**2)
        + 2 * (np.sqrt((-u[22] + u[44] + 1)**2 + (-u[23] + u[45] + 1)**2) - np.sqrt(2)) * (u[23] - u[45] - 1) / np.sqrt((-u[22] + u[44] + 1)**2 + (-u[23] + u[45] + 1)**2)
        + 2 * (np.sqrt((-u[22] + u[4] + 1)**2 + (u[23] - u[5] + 1)**2) - np.sqrt(2)) * (u[23] - u[5] + 1) / np.sqrt((-u[22] + u[4] + 1)**2 + (u[23] - u[5] + 1)**2)
        + 2 * (np.sqrt((u[22] - u[40] + 1)**2 + (-u[23] + u[41] + 1)**2) - np.sqrt(2)) * (u[23] - u[41] - 1) / np.sqrt((u[22] - u[40] + 1)**2 + (-u[23] + u[41] + 1)**2)
        + 2 * (np.sqrt((u[22] - u[2])**2 + (u[23] - u[3] + 1)**2) - 1) * (u[23] - u[3] + 1) / np.sqrt((u[22] - u[2])**2 + (u[23] - u[3] + 1)**2)
        + 2 * (np.sqrt((-u[22] + u[42])**2 + (-u[23] + u[43] + 1)**2) - 1) * (u[23] - u[43] - 1) / np.sqrt((-u[22] + u[42])**2 + (-u[23] + u[43] + 1)**2)
    )

    ans[24] = (
        -g[24]
        + 2 * (u[24] - u[44]) * (np.sqrt((-u[24] + u[44])**2 + (-u[25] + u[45] + 1)**2) - 1) / np.sqrt((-u[24] + u[44])**2 + (-u[25] + u[45] + 1)**2)
        + 2 * (u[24] - u[4]) * (np.sqrt((u[24] - u[4])**2 + (u[25] - u[5] + 1)**2) - 1) / np.sqrt((u[24] - u[4])**2 + (u[25] - u[5] + 1)**2)
        + 2 * (np.sqrt((-u[24] + u[46] + 1)**2 + (-u[25] + u[47] + 1)**2) - np.sqrt(2)) * (u[24] - u[46] - 1) / np.sqrt((-u[24] + u[46] + 1)**2 + (-u[25] + u[47] + 1)**2)
        + 2 * (np.sqrt((-u[24] + u[6] + 1)**2 + (u[25] - u[7] + 1)**2) - np.sqrt(2)) * (u[24] - u[6] - 1) / np.sqrt((-u[24] + u[6] + 1)**2 + (u[25] - u[7] + 1)**2)
        + 2 * (np.sqrt((u[24] - u[2] + 1)**2 + (u[25] - u[3] + 1)**2) - np.sqrt(2)) * (u[24] - u[2] + 1) / np.sqrt((u[24] - u[2] + 1)**2 + (u[25] - u[3] + 1)**2)
        + 2 * (np.sqrt((u[24] - u[42] + 1)**2 + (-u[25] + u[43] + 1)**2) - np.sqrt(2)) * (u[24] - u[42] + 1) / np.sqrt((u[24] - u[42] + 1)**2 + (-u[25] + u[43] + 1)**2)
        + 2 * (np.sqrt((-u[25] + u[27])**2 + (-u[24] + u[26] + 1)**2) - 1) * (u[24] - u[26] - 1) / np.sqrt((-u[25] + u[27])**2 + (-u[24] + u[26] + 1)**2)
        + 2 * (np.sqrt((-u[23] + u[25])**2 + (-u[22] + u[24] + 1)**2) - 1) * (-u[22] + u[24] + 1) / np.sqrt((-u[23] + u[25])**2 + (-u[22] + u[24] + 1)**2)
    )

    ans[25] = (
        -g[25]
        + 2 * (-u[23] + u[25]) * (np.sqrt((-u[23] + u[25])**2 + (-u[22] + u[24] + 1)**2) - 1) / np.sqrt((-u[23] + u[25])**2 + (-u[22] + u[24] + 1)**2)
        + 2 * (u[25] - u[27]) * (np.sqrt((-u[25] + u[27])**2 + (-u[24] + u[26] + 1)**2) - 1) / np.sqrt((-u[25] + u[27])**2 + (-u[24] + u[26] + 1)**2)
        + 2 * (np.sqrt((-u[24] + u[46] + 1)**2 + (-u[25] + u[47] + 1)**2) - np.sqrt(2)) * (u[25] - u[47] - 1) / np.sqrt((-u[24] + u[46] + 1)**2 + (-u[25] + u[47] + 1)**2)
        + 2 * (np.sqrt((-u[24] + u[6] + 1)**2 + (u[25] - u[7] + 1)**2) - np.sqrt(2)) * (u[25] - u[7] + 1) / np.sqrt((-u[24] + u[6] + 1)**2 + (u[25] - u[7] + 1)**2)
        + 2 * (np.sqrt((u[24] - u[2] + 1)**2 + (u[25] - u[3] + 1)**2) - np.sqrt(2)) * (u[25] - u[3] + 1) / np.sqrt((u[24] - u[2] + 1)**2 + (u[25] - u[3] + 1)**2)
        + 2 * (np.sqrt((u[24] - u[42] + 1)**2 + (-u[25] + u[43] + 1)**2) - np.sqrt(2)) * (u[25] - u[43] - 1) / np.sqrt((u[24] - u[42] + 1)**2 + (-u[25] + u[43] + 1)**2)
        + 2 * (np.sqrt((u[24] - u[4])**2 + (u[25] - u[5] + 1)**2) - 1) * (u[25] - u[5] + 1) / np.sqrt((u[24] - u[4])**2 + (u[25] - u[5] + 1)**2)
        + 2 * (np.sqrt((-u[24] + u[44])**2 + (-u[25] + u[45] + 1)**2) - 1) * (u[25] - u[45] - 1) / np.sqrt((-u[24] + u[44])**2 + (-u[25] + u[45] + 1)**2)
    )

    ans[26] = (
        -g[26]
        + 2 * (u[26] - u[46]) * (np.sqrt((-u[26] + u[46])**2 + (-u[27] + u[47] + 1)**2) - 1) / np.sqrt((-u[26] + u[46])**2 + (-u[27] + u[47] + 1)**2)
        + 2 * (u[26] - u[6]) * (np.sqrt((u[26] - u[6])**2 + (u[27] - u[7] + 1)**2) - 1) / np.sqrt((u[26] - u[6])**2 + (u[27] - u[7] + 1)**2)
        + 2 * (np.sqrt((-u[9] + u[27] + 1)**2 + (-u[26] + u[8] + 1)**2) - np.sqrt(2)) * (u[26] - u[8] - 1) / np.sqrt((-u[9] + u[27] + 1)**2 + (-u[26] + u[8] + 1)**2)
        + 2 * (np.sqrt((-u[26] + u[48] + 1)**2 + (-u[27] + u[49] + 1)**2) - np.sqrt(2)) * (u[26] - u[48] - 1) / np.sqrt((-u[26] + u[48] + 1)**2 + (-u[27] + u[49] + 1)**2)
        + 2 * (np.sqrt((u[26] - u[44] + 1)**2 + (-u[27] + u[45] + 1)**2) - np.sqrt(2)) * (u[26] - u[44] + 1) / np.sqrt((u[26] - u[44] + 1)**2 + (-u[27] + u[45] + 1)**2)
        + 2 * (np.sqrt((u[26] - u[4] + 1)**2 + (u[27] - u[5] + 1)**2) - np.sqrt(2)) * (u[26] - u[4] + 1) / np.sqrt((u[26] - u[4] + 1)**2 + (u[27] - u[5] + 1)**2)
        + 2 * (np.sqrt((-u[27] + u[29])**2 + (-u[26] + u[28] + 1)**2) - 1) * (u[26] - u[28] - 1) / np.sqrt((-u[27] + u[29])**2 + (-u[26] + u[28] + 1)**2)
        + 2 * (np.sqrt((-u[25] + u[27])**2 + (-u[24] + u[26] + 1)**2) - 1) * (-u[24] + u[26] + 1) / np.sqrt((-u[25] + u[27])**2 + (-u[24] + u[26] + 1)**2)
    )

    ans[27] = (
        -g[27]
        + 2 * (-u[25] + u[27]) * (np.sqrt((-u[25] + u[27])**2 + (-u[24] + u[26] + 1)**2) - 1) / np.sqrt((-u[25] + u[27])**2 + (-u[24] + u[26] + 1)**2)
        + 2 * (u[27] - u[29]) * (np.sqrt((-u[27] + u[29])**2 + (-u[26] + u[28] + 1)**2) - 1) / np.sqrt((-u[27] + u[29])**2 + (-u[26] + u[28] + 1)**2)
        + 2 * (np.sqrt((-u[9] + u[27] + 1)**2 + (-u[26] + u[8] + 1)**2) - np.sqrt(2)) * (-u[9] + u[27] + 1) / np.sqrt((-u[9] + u[27] + 1)**2 + (-u[26] + u[8] + 1)**2)
        + 2 * (np.sqrt((-u[26] + u[49] + 1)**2 + (-u[27] + u[50] + 1)**2) - np.sqrt(2)) * (u[27] - u[50] - 1) / np.sqrt((-u[26] + u[49] + 1)**2 + (-u[27] + u[50] + 1)**2)
        + 2 * (np.sqrt((u[26] - u[45] + 1)**2 + (-u[27] + u[46] + 1)**2) - np.sqrt(2)) * (u[27] - u[46] - 1) / np.sqrt((u[26] - u[45] + 1)**2 + (-u[27] + u[46] + 1)**2)
        + 2 * (np.sqrt((u[26] - u[4] + 1)**2 + (u[27] - u[5] + 1)**2) - np.sqrt(2)) * (u[27] - u[5] + 1) / np.sqrt((u[26] - u[4] + 1)**2 + (u[27] - u[5] + 1)**2)
        + 2 * (np.sqrt((u[26] - u[6])**2 + (u[27] - u[7] + 1)**2) - 1) * (u[27] - u[7] + 1) / np.sqrt((u[26] - u[6])**2 + (u[27] - u[7] + 1)**2)
        + 2 * (np.sqrt((-u[26] + u[46])**2 + (-u[27] + u[47] + 1)**2) - 1) * (u[27] - u[47] - 1) / np.sqrt((-u[26] + u[46])**2 + (-u[27] + u[47] + 1)**2)
    )

    ans[28] = (
        -g[28]
        + 2 * (u[28] - u[48]) * (np.sqrt((-u[28] + u[48])**2 + (-u[29] + u[49] + 1)**2) - 1) / np.sqrt((-u[28] + u[48])**2 + (-u[29] + u[49] + 1)**2)
        + 2 * (u[28] - u[8]) * (np.sqrt((u[28] - u[8])**2 + (-u[9] + u[29] + 1)**2) - 1) / np.sqrt((u[28] - u[8])**2 + (-u[9] + u[29] + 1)**2)
        + 2 * (np.sqrt((u[10] - u[28] + 1)**2 + (-u[11] + u[29] + 1)**2) - np.sqrt(2)) * (-u[10] + u[28] - 1) / np.sqrt((u[10] - u[28] + 1)**2 + (-u[11] + u[29] + 1)**2)
        + 2 * (np.sqrt((-u[28] + u[50] + 1)**2 + (-u[29] + u[51] + 1)**2) - np.sqrt(2)) * (u[28] - u[50] - 1) / np.sqrt((-u[28] + u[50] + 1)**2 + (-u[29] + u[51] + 1)**2)
        + 2 * (np.sqrt((u[28] - u[46] + 1)**2 + (-u[29] + u[47] + 1)**2) - np.sqrt(2)) * (u[28] - u[46] + 1) / np.sqrt((u[28] - u[46] + 1)**2 + (-u[29] + u[47] + 1)**2)
        + 2 * (np.sqrt((u[28] - u[6] + 1)**2 + (u[29] - u[7] + 1)**2) - np.sqrt(2)) * (u[28] - u[6] + 1) / np.sqrt((u[28] - u[6] + 1)**2 + (u[29] - u[7] + 1)**2)
        + 2 * (np.sqrt((-u[29] + u[31])**2 + (-u[28] + u[30] + 1)**2) - 1) * (u[28] - u[30] - 1) / np.sqrt((-u[29] + u[31])**2 + (-u[28] + u[30] + 1)**2)
        + 2 * (np.sqrt((-u[27] + u[29])**2 + (-u[26] + u[28] + 1)**2) - 1) * (-u[26] + u[28] + 1) / np.sqrt((-u[27] + u[29])**2 + (-u[26] + u[28] + 1)**2)
    )

    ans[29] = (
        -g[29]
        + 2 * (-u[27] + u[29]) * (np.sqrt((-u[27] + u[29])**2 + (-u[28] + u[28] + 1)**2) - 1) / np.sqrt((-u[27] + u[29])**2 + (-u[28] + u[28] + 1)**2)
        + 2 * (u[29] - u[31]) * (np.sqrt((-u[29] + u[31])**2 + (-u[28] + u[30] + 1)**2) - 1) / np.sqrt((-u[29] + u[31])**2 + (-u[28] + u[30] + 1)**2)
        + 2 * (np.sqrt((u[10] - u[28] + 1)**2 + (-u[11] + u[29] + 1)**2) - np.sqrt(2)) * (-u[11] + u[29] + 1) / np.sqrt((u[10] - u[28] + 1)**2 + (-u[11] + u[29] + 1)**2)
        + 2 * (np.sqrt((-u[28] + u[51] + 1)**2 + (-u[29] + u[52] + 1)**2) - np.sqrt(2)) * (u[29] - u[52] - 1) / np.sqrt((-u[28] + u[51] + 1)**2 + (-u[29] + u[52] + 1)**2)
        + 2 * (np.sqrt((u[28] - u[46] + 1)**2 + (-u[29] + u[47] + 1)**2) - np.sqrt(2)) * (u[29] - u[47] - 1) / np.sqrt((u[28] - u[46] + 1)**2 + (-u[29] + u[47] + 1)**2)
        + 2 * (np.sqrt((u[28] - u[6] + 1)**2 + (u[29] - u[7] + 1)**2) - np.sqrt(2)) * (u[29] - u[7] + 1) / np.sqrt((u[28] - u[6] + 1)**2 + (u[29] - u[7] + 1)**2)
        + 2 * (np.sqrt((u[28] - u[8])**2 + (-u[9] + u[29] + 1)**2) - 1) * (-u[9] + u[29] + 1) / np.sqrt((u[28] - u[8])**2 + (-u[9] + u[29] + 1)**2)
        + 2 * (np.sqrt((-u[28] + u[49])**2 + (-u[29] + u[50] + 1)**2) - 1) * (u[29] - u[50] - 1) / np.sqrt((-u[28] + u[49])**2 + (-u[29] + u[50] + 1)**2)
    )

    ans[30] = (
        -g[30]
        + 2 * (-u[10] + u[30]) * (np.sqrt((-u[10] + u[30])**2 + (-u[11] + u[31] + 1)**2) - 1) / np.sqrt((-u[10] + u[30])**2 + (-u[11] + u[31] + 1)**2)
        + 2 * (u[30] - u[50]) * (np.sqrt((-u[30] + u[50])**2 + (-u[31] + u[51] + 1)**2) - 1) / np.sqrt((-u[30] + u[50])**2 + (-u[31] + u[51] + 1)**2)
        + 2 * (np.sqrt((-u[9] + u[31] + 1)**2 + (u[30] - u[8] + 1)**2) - np.sqrt(2)) * (u[30] - u[8] + 1) / np.sqrt((-u[9] + u[31] + 1)**2 + (u[30] - u[8] + 1)**2)
        + 2 * (np.sqrt((u[12] - u[30] + 1)**2 + (-u[13] + u[31] + 1)**2) - np.sqrt(2)) * (-u[12] + u[30] - 1) / np.sqrt((u[12] - u[30] + 1)**2 + (-u[13] + u[31] + 1)**2)
        + 2 * (np.sqrt((-u[30] + u[52] + 1)**2 + (-u[31] + u[53] + 1)**2) - np.sqrt(2)) * (u[30] - u[52] - 1) / np.sqrt((-u[30] + u[52] + 1)**2 + (-u[31] + u[53] + 1)**2)
        + 2 * (np.sqrt((u[30] - u[48] + 1)**2 + (-u[31] + u[49] + 1)**2) - np.sqrt(2)) * (u[30] - u[48] + 1) / np.sqrt((u[30] - u[48] + 1)**2 + (-u[31] + u[49] + 1)**2)
        + 2 * (np.sqrt((-u[31] + u[33])**2 + (-u[30] + u[32] + 1)**2) - 1) * (u[30] - u[32] - 1) / np.sqrt((-u[31] + u[33])**2 + (-u[30] + u[32] + 1)**2)
        + 2 * (np.sqrt((-u[29] + u[31])**2 + (-u[28] + u[30] + 1)**2) - 1) * (-u[28] + u[30] + 1) / np.sqrt((-u[29] + u[31])**2 + (-u[28] + u[30] + 1)**2)
    )

    ans[31] = (
        -g[31]
        + 2 * (-u[29] + u[31]) * (np.sqrt((-u[29] + u[31])**2 + (-u[30] + u[32] + 1)**2) - 1) / np.sqrt((-u[29] + u[31])**2 + (-u[30] + u[32] + 1)**2)
        + 2 * (u[31] - u[33]) * (np.sqrt((-u[31] + u[33])**2 + (-u[30] + u[32] + 1)**2) - 1) / np.sqrt((-u[31] + u[33])**2 + (-u[30] + u[32] + 1)**2)
        + 2 * (np.sqrt((-u[9] + u[31] + 1)**2 + (u[30] - u[8] + 1)**2) - np.sqrt(2)) * (-u[9] + u[31] + 1) / np.sqrt((-u[9] + u[31] + 1)**2 + (u[30] - u[8] + 1)**2)
        + 2 * (np.sqrt((u[12] - u[30] + 1)**2 + (-u[13] + u[31] + 1)**2) - np.sqrt(2)) * (-u[13] + u[31] + 1) / np.sqrt((u[12] - u[30] + 1)**2 + (-u[13] + u[31] + 1)**2)
        + 2 * (np.sqrt((-u[30] + u[52] + 1)**2 + (-u[31] + u[53] + 1)**2) - np.sqrt(2)) * (u[31] - u[53] - 1) / np.sqrt((-u[30] + u[52] + 1)**2 + (-u[31] + u[53] + 1)**2)
        + 2 * (np.sqrt((u[30] - u[48] + 1)**2 + (-u[31] + u[49] + 1)**2) - np.sqrt(2)) * (u[31] - u[49] - 1) / np.sqrt((u[30] - u[48] + 1)**2 + (-u[31] + u[49] + 1)**2)
        + 2 * (np.sqrt((-u[30] + u[50])**2 + (-u[31] + u[51] + 1)**2) - 1) * (u[31] - u[51] - 1) / np.sqrt((-u[30] + u[50])**2 + (-u[31] + u[51] + 1)**2)
        + 2 * (np.sqrt((-u[10] + u[30])**2 + (-u[11] + u[31] + 1)**2) - 1) * (-u[11] + u[31] + 1) / np.sqrt((-u[10] + u[30])**2 + (-u[11] + u[31] + 1)**2)
    )

    ans[32] = (
        -g[32]
        + 2 * (-u[12] + u[32]) * (np.sqrt((-u[12] + u[32])**2 + (-u[13] + u[33] + 1)**2) - 1) / np.sqrt((-u[12] + u[32])**2 + (-u[13] + u[33] + 1)**2)
        + 2 * (u[32] - u[52]) * (np.sqrt((-u[32] + u[52])**2 + (-u[33] + u[53] + 1)**2) - 1) / np.sqrt((-u[32] + u[52])**2 + (-u[33] + u[53] + 1)**2)
        + 2 * (np.sqrt((-u[10] + u[32] + 1)**2 + (-u[11] + u[33] + 1)**2) - np.sqrt(2)) * (-u[10] + u[32] + 1) / np.sqrt((-u[10] + u[32] + 1)**2 + (-u[11] + u[33] + 1)**2)
        + 2 * (np.sqrt((u[14] - u[32] + 1)**2 + (-u[15] + u[33] + 1)**2) - np.sqrt(2)) * (-u[15] + u[32] - 1) / np.sqrt((u[14] - u[32] + 1)**2 + (-u[15] + u[33] + 1)**2)
        + 2 * (np.sqrt((-u[32] + u[54] + 1)**2 + (-u[33] + u[55] + 1)**2) - np.sqrt(2)) * (u[32] - u[54] - 1) / np.sqrt((-u[32] + u[54] + 1)**2 + (-u[33] + u[55] + 1)**2)
        + 2 * (np.sqrt((u[32] - u[50] + 1)**2 + (-u[33] + u[51] + 1)**2) - np.sqrt(2)) * (u[32] - u[50] + 1) / np.sqrt((u[32] - u[50] + 1)**2 + (-u[33] + u[51] + 1)**2)
        + 2 * (np.sqrt((-u[33] + u[35])**2 + (-u[32] + u[34] + 1)**2) - 1) * (u[32] - u[34] - 1) / np.sqrt((-u[33] + u[35])**2 + (-u[32] + u[34] + 1)**2)
        + 2 * (np.sqrt((-u[31] + u[33])**2 + (-u[30] + u[32] + 1)**2) - 1) * (-u[30] + u[32] + 1) / np.sqrt((-u[31] + u[33])**2 + (-u[30] + u[32] + 1)**2)
    )

    ans[33] = (
        -g[33]
        + 2 * (-u[31] + u[33]) * (np.sqrt((-u[31] + u[33])**2 + (-u[30] + u[32] + 1)**2) - 1) / np.sqrt((-u[31] + u[33])**2 + (-u[30] + u[32] + 1)**2)
        + 2 * (u[33] - u[35]) * (np.sqrt((-u[33] + u[35])**2 + (-u[32] + u[34] + 1)**2) - 1) / np.sqrt((-u[33] + u[35])**2 + (-u[32] + u[34] + 1)**2)
        + 2 * (np.sqrt((-u[11] + u[32] + 1)**2 + (-u[12] + u[33] + 1)**2) - np.sqrt(2)) * (-u[12] + u[33] + 1) / np.sqrt((-u[11] + u[32] + 1)**2 + (-u[12] + u[33] + 1)**2)
        + 2 * (np.sqrt((u[14] - u[32] + 1)**2 + (-u[15] + u[33] + 1)**2) - np.sqrt(2)) * (-u[15] + u[33] + 1) / np.sqrt((u[14] - u[32] + 1)**2 + (-u[15] + u[33] + 1)**2)
        + 2 * (np.sqrt((-u[32] + u[54] + 1)**2 + (-u[33] + u[55] + 1)**2) - np.sqrt(2)) * (u[33] - u[55] - 1) / np.sqrt((-u[32] + u[54] + 1)**2 + (-u[33] + u[55] + 1)**2)
        + 2 * (np.sqrt((u[32] - u[50] + 1)**2 + (-u[33] + u[51] + 1)**2) - np.sqrt(2)) * (u[33] - u[51] - 1) / np.sqrt((u[32] - u[50] + 1)**2 + (-u[33] + u[51] + 1)**2)
        + 2 * (np.sqrt((-u[32] + u[52])**2 + (-u[33] + u[53] + 1)**2) - 1) * (u[33] - u[53] - 1) / np.sqrt((-u[32] + u[52])**2 + (-u[33] + u[53] + 1)**2)
        + 2 * (np.sqrt((-u[12] + u[32])**2 + (-u[13] + u[33] + 1)**2) - 1) * (-u[13] + u[33] + 1) / np.sqrt((-u[12] + u[32])**2 + (-u[13] + u[33] + 1)**2)
    )

    ans[34] = (
        -g[34]
        + 2 * (-u[14] + u[34]) * (np.sqrt((-u[14] + u[34])**2 + (-u[15] + u[35] + 1)**2) - 1) / np.sqrt((-u[14] + u[34])**2 + (-u[15] + u[35] + 1)**2)
        + 2 * (u[34] - u[54]) * (np.sqrt((-u[34] + u[54])**2 + (-u[35] + u[55] + 1)**2) - 1) / np.sqrt((-u[34] + u[54])**2 + (-u[35] + u[55] + 1)**2)
        + 2 * (np.sqrt((-u[12] + u[34] + 1)**2 + (-u[13] + u[35] + 1)**2) - np.sqrt(2)) * (-u[12] + u[34] + 1) / np.sqrt((-u[12] + u[34] + 1)**2 + (-u[13] + u[35] + 1)**2)
        + 2 * (np.sqrt((u[16] - u[34] + 1)**2 + (-u[17] + u[35] + 1)**2) - np.sqrt(2)) * (-u[17] + u[34] - 1) / np.sqrt((u[16] - u[34] + 1)**2 + (-u[17] + u[35] + 1)**2)
        + 2 * (np.sqrt((-u[34] + u[56] + 1)**2 + (-u[35] + u[57] + 1)**2) - np.sqrt(2)) * (u[34] - u[56] - 1) / np.sqrt((-u[34] + u[56] + 1)**2 + (-u[35] + u[57] + 1)**2)
        + 2 * (np.sqrt((u[34] - u[52] + 1)**2 + (-u[35] + u[53] + 1)**2) - np.sqrt(2)) * (u[34] - u[52] + 1) / np.sqrt((u[34] - u[52] + 1)**2 + (-u[35] + u[53] + 1)**2)
        + 2 * (np.sqrt((-u[35] + u[37])**2 + (-u[34] + u[36] + 1)**2) - 1) * (u[34] - u[36] - 1) / np.sqrt((-u[35] + u[37])**2 + (-u[34] + u[36] + 1)**2)
        + 2 * (np.sqrt((-u[33] + u[35])**2 + (-u[32] + u[34] + 1)**2) - 1) * (-u[32] + u[34] + 1) / np.sqrt((-u[33] + u[35])**2 + (-u[32] + u[34] + 1)**2)
    )

    ans[35] = (
        -g[35]
        + 2 * (-u[33] + u[35]) * (np.sqrt((-u[33] + u[35])**2 + (-u[32] + u[34] + 1)**2) - 1) / np.sqrt((-u[33] + u[35])**2 + (-u[32] + u[34] + 1)**2)
        + 2 * (u[35] - u[37]) * (np.sqrt((-u[35] + u[37])**2 + (-u[34] + u[36] + 1)**2) - 1) / np.sqrt((-u[35] + u[37])**2 + (-u[34] + u[36] + 1)**2)
        + 2 * (np.sqrt((-u[12] + u[34] + 1)**2 + (-u[13] + u[35] + 1)**2) - np.sqrt(2)) * (-u[13] + u[35] + 1) / np.sqrt((-u[12] + u[34] + 1)**2 + (-u[13] + u[35] + 1)**2)
        + 2 * (np.sqrt((u[16] - u[34] + 1)**2 + (-u[17] + u[35] + 1)**2) - np.sqrt(2)) * (-u[17] + u[35] + 1) / np.sqrt((u[16] - u[34] + 1)**2 + (-u[17] + u[35] + 1)**2)
        + 2 * (np.sqrt((-u[34] + u[56] + 1)**2 + (-u[35] + u[57] + 1)**2) - np.sqrt(2)) * (u[35] - u[57] - 1) / np.sqrt((-u[34] + u[56] + 1)**2 + (-u[35] + u[57] + 1)**2)
        + 2 * (np.sqrt((u[34] - u[52] + 1)**2 + (-u[35] + u[53] + 1)**2) - np.sqrt(2)) * (u[35] - u[53] - 1) / np.sqrt((u[34] - u[52] + 1)**2 + (-u[35] + u[53] + 1)**2)
        + 2 * (np.sqrt((-u[34] + u[54])**2 + (-u[35] + u[55] + 1)**2) - 1) * (u[35] - u[55] - 1) / np.sqrt((-u[34] + u[54])**2 + (-u[35] + u[55] + 1)**2)
        + 2 * (np.sqrt((-u[14] + u[34])**2 + (-u[15] + u[35] + 1)**2) - 1) * (-u[15] + u[35] + 1) / np.sqrt((-u[14] + u[34])**2 + (-u[15] + u[35] + 1)**2)
    )

    ans[36] = (
        -g[36]
        + 2 * (-u[16] + u[36]) * (np.sqrt((-u[16] + u[36])**2 + (-u[17] + u[37] + 1)**2) - 1) / np.sqrt((-u[16] + u[36])**2 + (-u[17] + u[37] + 1)**2)
        + 2 * (u[36] - u[56]) * (np.sqrt((-u[36] + u[56])**2 + (-u[37] + u[57] + 1)**2) - 1) / np.sqrt((-u[36] + u[56])**2 + (-u[37] + u[57] + 1)**2)
        + 2 * (np.sqrt((-u[14] + u[36] + 1)**2 + (-u[15] + u[37] + 1)**2) - np.sqrt(2)) * (-u[14] + u[36] + 1) / np.sqrt((-u[14] + u[36] + 1)**2 + (-u[15] + u[37] + 1)**2)
        + 2 * (np.sqrt((u[18] - u[36] + 1)**2 + (-u[19] + u[37] + 1)**2) - np.sqrt(2)) * (-u[18] + u[36] - 1) / np.sqrt((u[18] - u[36] + 1)**2 + (-u[19] + u[37] + 1)**2)
        + 2 * (np.sqrt((-u[36] + u[58] + 1)**2 + (-u[37] + u[59] + 1)**2) - np.sqrt(2)) * (u[36] - u[58] - 1) / np.sqrt((-u[36] + u[58] + 1)**2 + (-u[37] + u[59] + 1)**2)
        + 2 * (np.sqrt((u[36] - u[54] + 1)**2 + (-u[37] + u[55] + 1)**2) - np.sqrt(2)) * (u[36] - u[54] + 1) / np.sqrt((u[36] - u[54] + 1)**2 + (-u[37] + u[55] + 1)**2)
        + 2 * (np.sqrt((-u[37] + u[39])**2 + (-u[36] + u[38] + 1)**2) - 1) * (u[36] - u[38] - 1) / np.sqrt((-u[37] + u[39])**2 + (-u[36] + u[38] + 1)**2)
        + 2 * (np.sqrt((-u[35] + u[37])**2 + (-u[34] + u[36] + 1)**2) - 1) * (-u[34] + u[36] + 1) / np.sqrt((-u[35] + u[37])**2 + (-u[34] + u[36] + 1)**2)
    )

    ans[37] = (
        -g[37]
        + 2 * (-u[35] + u[37]) * (np.sqrt((-u[35] + u[37])**2 + (-u[34] + u[36] + 1)**2) - 1) / np.sqrt((-u[35] + u[37])**2 + (-u[34] + u[36] + 1)**2)
        + 2 * (u[37] - u[39]) * (np.sqrt((-u[37] + u[39])**2 + (-u[36] + u[38] + 1)**2) - 1) / np.sqrt((-u[37] + u[39])**2 + (-u[36] + u[38] + 1)**2)
        + 2 * (np.sqrt((-u[14] + u[36] + 1)**2 + (-u[15] + u[37] + 1)**2) - np.sqrt(2)) * (-u[15] + u[37] + 1) / np.sqrt((-u[14] + u[36] + 1)**2 + (-u[15] + u[37] + 1)**2)
        + 2 * (np.sqrt((u[18] - u[36] + 1)**2 + (-u[19] + u[37] + 1)**2) - np.sqrt(2)) * (-u[19] + u[37] + 1) / np.sqrt((u[18] - u[36] + 1)**2 + (-u[19] + u[37] + 1)**2)
        + 2 * (np.sqrt((-u[36] + u[59] + 1)**2 + (-u[37] + u[60] + 1)**2) - np.sqrt(2)) * (u[37] - u[59] - 1) / np.sqrt((-u[36] + u[59] + 1)**2 + (-u[37] + u[60] + 1)**2)
        + 2 * (np.sqrt((u[36] - u[55] + 1)**2 + (-u[37] + u[56] + 1)**2) - np.sqrt(2)) * (u[37] - u[56] - 1) / np.sqrt((u[36] - u[55] + 1)**2 + (-u[37] + u[56] + 1)**2)
        + 2 * (np.sqrt((-u[36] + u[57])**2 + (-u[37] + u[58] + 1)**2) - 1) * (u[37] - u[58] - 1) / np.sqrt((-u[36] + u[57])**2 + (-u[37] + u[58] + 1)**2)
        + 2 * (np.sqrt((-u[16] + u[36])**2 + (-u[17] + u[37] + 1)**2) - 1) * (-u[17] + u[37] + 1) / np.sqrt((-u[16] + u[36])**2 + (-u[17] + u[37] + 1)**2)
    )

    ans[38] = (
        -g[38]
        + 2 * (-u[18] + u[38]) * (np.sqrt((-u[18] + u[38])**2 + (-u[19] + u[39] + 1)**2) - 1) / np.sqrt((-u[18] + u[38])**2 + (-u[19] + u[39] + 1)**2)
        + 2 * (u[38] - u[58]) * (np.sqrt((-u[38] + u[58])**2 + (-u[39] + u[59] + 1)**2) - 1) / np.sqrt((-u[38] + u[58])**2 + (-u[39] + u[59] + 1)**2)
        + 2 * (np.sqrt((-u[16] + u[38] + 1)**2 + (-u[17] + u[39] + 1)**2) - np.sqrt(2)) * (-u[16] + u[38] + 1) / np.sqrt((-u[16] + u[38] + 1)**2 + (-u[17] + u[39] + 1)**2)
        + 2 * (np.sqrt((u[38] - u[56] + 1)**2 + (-u[39] + u[57] + 1)**2) - np.sqrt(2)) * (u[38] - u[56] + 1) / np.sqrt((u[38] - u[56] + 1)**2 + (-u[39] + u[57] + 1)**2)
        + 2 * (np.sqrt((-u[37] + u[39])**2 + (-u[36] + u[38] + 1)**2) - 1) * (-u[36] + u[38] + 1) / np.sqrt((-u[37] + u[39])**2 + (-u[36] + u[38] + 1)**2)
    )

    ans[39] = (
        -g[39]
        + 2 * (-u[37] + u[39]) * (np.sqrt((-u[37] + u[39])**2 + (-u[36] + u[38] + 1)**2) - 1) / np.sqrt((-u[37] + u[39])**2 + (-u[36] + u[38] + 1)**2)
        + 2 * (np.sqrt((-u[16] + u[38] + 1)**2 + (-u[17] + u[39] + 1)**2) - np.sqrt(2)) * (-u[17] + u[39] + 1) / np.sqrt((-u[16] + u[38] + 1)**2 + (-u[17] + u[39] + 1)**2)
        + 2 * (np.sqrt((u[38] - u[56] + 1)**2 + (-u[39] + u[57] + 1)**2) - np.sqrt(2)) * (u[39] - u[57] - 1) / np.sqrt((u[38] - u[56] + 1)**2 + (-u[39] + u[57] + 1)**2)
        + 2 * (np.sqrt((-u[38] + u[58])**2 + (-u[39] + u[59] + 1)**2) - 1) * (u[39] - u[59] - 1) / np.sqrt((-u[38] + u[58])**2 + (-u[39] + u[59] + 1)**2)
        + 2 * (np.sqrt((-u[18] + u[38])**2 + (-u[19] + u[39] + 1)**2) - 1) * (-u[19] + u[39] + 1) / np.sqrt((-u[18] + u[38])**2 + (-u[19] + u[39] + 1)**2)
    )

    ans[40] = (
        -g[40]
        + 2 * (-u[20] + u[40]) * (np.sqrt((-u[20] + u[40])**2 + (-u[21] + u[41] + 1)**2) - 1) / np.sqrt((-u[20] + u[40])**2 + (-u[21] + u[41] + 1)**2)
        + 2 * (u[40] - u[60]) * (np.sqrt((-u[40] + u[60])**2 + (-u[41] + u[61] + 1)**2) - 1) / np.sqrt((-u[40] + u[60])**2 + (-u[41] + u[61] + 1)**2)
        + 2 * (np.sqrt((u[22] - u[40] + 1)**2 + (-u[23] + u[41] + 1)**2) - np.sqrt(2)) * (-u[22] + u[40] - 1) / np.sqrt((u[22] - u[40] + 1)**2 + (-u[23] + u[41] + 1)**2)
        + 2 * (np.sqrt((-u[40] + u[62] + 1)**2 + (-u[41] + u[63] + 1)**2) - np.sqrt(2)) * (u[40] - u[62] - 1) / np.sqrt((-u[40] + u[62] + 1)**2 + (-u[41] + u[63] + 1)**2)
        + 2 * (np.sqrt((-u[21] + u[43])**2 + (-u[40] + u[42] + 1)**2) - 1) * (u[40] - u[42] - 1) / np.sqrt((-u[21] + u[43])**2 + (-u[40] + u[42] + 1)**2)
    )

    ans[41] = (
        -g[41]
        + 2 * (u[41] - u[43]) * (np.sqrt((-u[41] + u[43])**2 + (-u[40] + u[42] + 1)**2) - 1) / np.sqrt((-u[41] + u[43])**2 + (-u[40] + u[42] + 1)**2)
        + 2 * (np.sqrt((u[22] - u[40] + 1)**2 + (-u[23] + u[41] + 1)**2) - np.sqrt(2)) * (-u[23] + u[41] + 1) / np.sqrt((u[22] - u[40] + 1)**2 + (-u[23] + u[41] + 1)**2)
        + 2 * (np.sqrt((-u[40] + u[62] + 1)**2 + (-u[41] + u[63] + 1)**2) - np.sqrt(2)) * (u[41] - u[63] - 1) / np.sqrt((-u[40] + u[62] + 1)**2 + (-u[41] + u[63] + 1)**2)
        + 2 * (np.sqrt((-u[40] + u[60])**2 + (-u[41] + u[61] + 1)**2) - 1) * (u[41] - u[61] - 1) / np.sqrt((-u[40] + u[60])**2 + (-u[41] + u[61] + 1)**2)
        + 2 * (np.sqrt((-u[20] + u[40])**2 + (-u[21] + u[41] + 1)**2) - 1) * (-u[21] + u[41] + 1) / np.sqrt((-u[20] + u[40])**2 + (-u[21] + u[41] + 1)**2)
    )

    ans[42] = (
        -g[42]
        + 2 * ( -u[22] + u[42] ) * (np.sqrt((-u[22] + u[42])**2 + (-u[23] + u[43] + 1)**2) - 1) / np.sqrt((-u[22] + u[42])**2 + (-u[23] + u[43] + 1)**2)
        + 2 * ( u[42] - u[62] ) * (np.sqrt((-u[42] + u[62])**2 + (-u[43] + u[63] + 1)**2) - 1) / np.sqrt((-u[42] + u[62])**2 + (-u[43] + u[63] + 1)**2)
        + 2 * (np.sqrt((-u[20] + u[42] + 1)**2 + (-u[21] + u[43] + 1)**2) - np.sqrt(2)) * (-u[20] + u[42] + 1) / np.sqrt((-u[20] + u[42] + 1)**2 + (-u[21] + u[43] + 1)**2)
        + 2 * (np.sqrt((u[24] - u[42] + 1)**2 + (-u[25] + u[43] + 1)**2) - np.sqrt(2)) * (-u[24] + u[42] - 1) / np.sqrt((u[24] - u[42] + 1)**2 + (-u[25] + u[43] + 1)**2)
        + 2 * (np.sqrt((-u[42] + u[64] + 1)**2 + (-u[43] + u[65] + 1)**2) - np.sqrt(2)) * (u[42] - u[64] - 1) / np.sqrt((-u[42] + u[64] + 1)**2 + (-u[43] + u[65] + 1)**2)
        + 2 * (np.sqrt((u[42] - u[60] + 1)**2 + (-u[43] + u[61] + 1)**2) - np.sqrt(2)) * (u[42] - u[60] + 1) / np.sqrt((u[42] - u[60] + 1)**2 + (-u[43] + u[61] + 1)**2)
        + 2 * (np.sqrt((-u[43] + u[45])**2 + (-u[42] + u[44] + 1)**2) - 1) * (u[42] - u[44] - 1) / np.sqrt((-u[43] + u[45])**2 + (-u[42] + u[44] + 1)**2)
        + 2 * (np.sqrt((-u[41] + u[43])**2 + (-u[40] + u[42] + 1)**2) - 1) * (-u[40] + u[42] + 1) / np.sqrt((-u[41] + u[43])**2 + (-u[40] + u[42] + 1)**2)
    )

    ans[43] = (
        -g[43]
        + 2 * (-u[41] + u[43]) * (np.sqrt((-u[41] + u[43])**2 + (-u[40] + u[42] + 1)**2) - 1) / np.sqrt((-u[41] + u[43])**2 + (-u[40] + u[42] + 1)**2)
        + 2 * (u[43] - u[45]) * (np.sqrt((-u[43] + u[45])**2 + (-u[42] + u[44] + 1)**2) - 1) / np.sqrt((-u[43] + u[45])**2 + (-u[42] + u[44] + 1)**2)
        + 2 * (np.sqrt((-u[20] + u[42] + 1)**2 + (-u[21] + u[43] + 1)**2) - np.sqrt(2)) * (-u[21] + u[43] + 1) / np.sqrt((-u[20] + u[42] + 1)**2 + (-u[21] + u[43] + 1)**2)
        + 2 * (np.sqrt((u[24] - u[42] + 1)**2 + (-u[25] + u[43] + 1)**2) - np.sqrt(2)) * (-u[25] + u[43] + 1) / np.sqrt((u[24] - u[42] + 1)**2 + (-u[25] + u[43] + 1)**2)
        + 2 * (np.sqrt((-u[42] + u[64] + 1)**2 + (-u[43] + u[65] + 1)**2) - np.sqrt(2)) * (u[43] - u[65] - 1) / np.sqrt((-u[42] + u[64] + 1)**2 + (-u[43] + u[65] + 1)**2)
        + 2 * (np.sqrt((u[42] - u[60] + 1)**2 + (-u[43] + u[61] + 1)**2) - np.sqrt(2)) * (u[43] - u[61] - 1) / np.sqrt((u[42] - u[60] + 1)**2 + (-u[43] + u[61] + 1)**2)
        + 2 * (np.sqrt((-u[42] + u[62])**2 + (-u[43] + u[63] + 1)**2) - 1) * (u[43] - u[63] - 1) / np.sqrt((-u[42] + u[62])**2 + (-u[43] + u[63] + 1)**2)
        + 2 * (np.sqrt((-u[22] + u[42])**2 + (-u[23] + u[43] + 1)**2) - 1) * (-u[23] + u[43] + 1) / np.sqrt((-u[22] + u[42])**2 + (-u[23] + u[43] + 1)**2)
    )

    ans[44] = (
        -g[44]
        + 2 * (-u[24] + u[44]) * (np.sqrt((-u[24] + u[44])**2 + (-u[25] + u[45] + 1)**2) - 1) / np.sqrt((-u[24] + u[44])**2 + (-u[25] + u[45] + 1)**2)
        + 2 * (u[44] - u[64]) * (np.sqrt((-u[44] + u[64])**2 + (-u[45] + u[65] + 1)**2) - 1) / np.sqrt((-u[44] + u[64])**2 + (-u[45] + u[65] + 1)**2)
        + 2 * (np.sqrt((-u[22] + u[44] + 1)**2 + (-u[23] + u[45] + 1)**2) - np.sqrt(2)) * (-u[22] + u[44] + 1) / np.sqrt((-u[22] + u[44] + 1)**2 + (-u[23] + u[45] + 1)**2)
        + 2 * (np.sqrt((u[26] - u[44] + 1)**2 + (-u[27] + u[45] + 1)**2) - np.sqrt(2)) * (-u[26] + u[44] - 1) / np.sqrt((u[26] - u[44] + 1)**2 + (-u[27] + u[45] + 1)**2)
        + 2 * (np.sqrt((-u[44] + u[66] + 1)**2 + (-u[45] + u[67] + 1)**2) - np.sqrt(2)) * (u[44] - u[66] - 1) / np.sqrt((-u[44] + u[66] + 1)**2 + (-u[45] + u[67] + 1)**2)
        + 2 * (np.sqrt((u[44] - u[62] + 1)**2 + (-u[45] + u[63] + 1)**2) - np.sqrt(2)) * (u[44] - u[62] + 1) / np.sqrt((u[44] - u[62] + 1)**2 + (-u[45] + u[63] + 1)**2)
        + 2 * (np.sqrt((-u[45] + u[47])**2 + (-u[44] + u[46] + 1)**2) - 1) * (u[44] - u[46] - 1) / np.sqrt((-u[45] + u[47])**2 + (-u[44] + u[46] + 1)**2)
        + 2 * (np.sqrt((-u[43] + u[45])**2 + (-u[42] + u[44] + 1)**2) - 1) * (-u[42] + u[44] + 1) / np.sqrt((-u[43] + u[45])**2 + (-u[42] + u[44] + 1)**2)
    )

    ans[45] = (
        -g[45]
        + 2 * (-u[43] + u[45]) * (np.sqrt((-u[43] + u[45])**2 + (-u[42] + u[44] + 1)**2) - 1) / np.sqrt((-u[43] + u[45])**2 + (-u[42] + u[44] + 1)**2)
        + 2 * (u[45] - u[47]) * (np.sqrt((-u[45] + u[47])**2 + (-u[44] + u[46] + 1)**2) - 1) / np.sqrt((-u[45] + u[47])**2 + (-u[44] + u[46] + 1)**2)
        + 2 * (np.sqrt((-u[22] + u[44] + 1)**2 + (-u[23] + u[45] + 1)**2) - np.sqrt(2)) * (-u[23] + u[45] + 1) / np.sqrt((-u[22] + u[44] + 1)**2 + (-u[23] + u[45] + 1)**2)
        + 2 * (np.sqrt((u[26] - u[44] + 1)**2 + (-u[27] + u[45] + 1)**2) - np.sqrt(2)) * (-u[27] + u[45] + 1) / np.sqrt((u[26] - u[44] + 1)**2 + (-u[27] + u[45] + 1)**2)
        + 2 * (np.sqrt((-u[44] + u[66] + 1)**2 + (-u[45] + u[67] + 1)**2) - np.sqrt(2)) * (u[45] - u[67] - 1) / np.sqrt((-u[44] + u[66] + 1)**2 + (-u[45] + u[67] + 1)**2)
        + 2 * (np.sqrt((u[44] - u[62] + 1)**2 + (-u[45] + u[63] + 1)**2) - np.sqrt(2)) * (u[45] - u[63] - 1) / np.sqrt((u[44] - u[62] + 1)**2 + (-u[45] + u[63] + 1)**2)
        + 2 * (np.sqrt((-u[44] + u[64])**2 + (-u[45] + u[65] + 1)**2) - 1) * (u[45] - u[65] - 1) / np.sqrt((-u[44] + u[64])**2 + (-u[45] + u[65] + 1)**2)
        + 2 * (np.sqrt((-u[24] + u[44])**2 + (-u[25] + u[45] + 1)**2) - 1) * (-u[25] + u[45] + 1) / np.sqrt((-u[24] + u[44])**2 + (-u[25] + u[45] + 1)**2)
    )

    ans[46] = (
        -g[46]
        + 2 * (-u[26] + u[46]) * (np.sqrt((-u[26] + u[46])**2 + (-u[27] + u[47] + 1)**2) - 1) / np.sqrt((-u[26] + u[46])**2 + (-u[27] + u[47] + 1)**2)
        + 2 * (u[46] - u[66]) * (np.sqrt((-u[46] + u[66])**2 + (-u[47] + u[67] + 1)**2) - 1) / np.sqrt((-u[46] + u[66])**2 + (-u[47] + u[67] + 1)**2)
        + 2 * (np.sqrt((-u[24] + u[46] + 1)**2 + (-u[25] + u[47] + 1)**2) - np.sqrt(2)) * (-u[24] + u[46] + 1) / np.sqrt((-u[24] + u[46] + 1)**2 + (-u[25] + u[47] + 1)**2)
        + 2 * (np.sqrt((u[28] - u[46] + 1)**2 + (-u[29] + u[47] + 1)**2) - np.sqrt(2)) * (-u[28] + u[46] - 1) / np.sqrt((u[28] - u[46] + 1)**2 + (-u[29] + u[47] + 1)**2)
        + 2 * (np.sqrt((-u[46] + u[68] + 1)**2 + (-u[47] + u[69] + 1)**2) - np.sqrt(2)) * (u[46] - u[68] - 1) / np.sqrt((-u[46] + u[68] + 1)**2 + (-u[47] + u[69] + 1)**2)
        + 2 * (np.sqrt((u[46] - u[64] + 1)**2 + (-u[47] + u[65] + 1)**2) - np.sqrt(2)) * (u[46] - u[64] + 1) / np.sqrt((u[46] - u[64] + 1)**2 + (-u[47] + u[65] + 1)**2)
        + 2 * (np.sqrt((-u[47] + u[49])**2 + (-u[46] + u[48] + 1)**2) - 1) * (u[46] - u[48] - 1) / np.sqrt((-u[47] + u[49])**2 + (-u[46] + u[48] + 1)**2)
        + 2 * (np.sqrt((-u[45] + u[47])**2 + (-u[44] + u[46] + 1)**2) - 1) * (-u[44] + u[46] + 1) / np.sqrt((-u[45] + u[47])**2 + (-u[44] + u[46] + 1)**2)
    )

    ans[47] = (
        -g[47]
        + 2 * (-u[45] + u[47]) * (np.sqrt((-u[45] + u[47])**2 + (-u[44] + u[46] + 1)**2) - 1) / np.sqrt((-u[45] + u[47])**2 + (-u[44] + u[46] + 1)**2)
        + 2 * (u[47] - u[49]) * (np.sqrt((-u[47] + u[49])**2 + (-u[46] + u[48] + 1)**2) - 1) / np.sqrt((-u[47] + u[49])**2 + (-u[46] + u[48] + 1)**2)
        + 2 * (np.sqrt((-u[24] + u[46] + 1)**2 + (-u[25] + u[47] + 1)**2) - np.sqrt(2)) * (-u[25] + u[47] + 1) / np.sqrt((-u[24] + u[46] + 1)**2 + (-u[25] + u[47] + 1)**2)
        + 2 * (np.sqrt((u[28] - u[46] + 1)**2 + (-u[29] + u[47] + 1)**2) - np.sqrt(2)) * (-u[29] + u[47] + 1) / np.sqrt((u[28] - u[46] + 1)**2 + (-u[29] + u[47] + 1)**2)
        + 2 * (np.sqrt((-u[46] + u[69] + 1)**2 + (-u[47] + u[70] + 1)**2) - np.sqrt(2)) * (u[47] - u[70] - 1) / np.sqrt((-u[46] + u[69] + 1)**2 + (-u[47] + u[70] + 1)**2)
        + 2 * (np.sqrt((u[46] - u[65] + 1)**2 + (-u[47] + u[66] + 1)**2) - np.sqrt(2)) * (u[47] - u[66] - 1) / np.sqrt((u[46] - u[65] + 1)**2 + (-u[47] + u[66] + 1)**2)
        + 2 * (np.sqrt((-u[46] + u[67])**2 + (-u[47] + u[68] + 1)**2) - 1) * (u[47] - u[68] - 1) / np.sqrt((-u[46] + u[67])**2 + (-u[47] + u[68] + 1)**2)
        + 2 * (np.sqrt((-u[26] + u[46])**2 + (-u[27] + u[47] + 1)**2) - 1) * (-u[27] + u[47] + 1) / np.sqrt((-u[26] + u[46])**2 + (-u[27] + u[47] + 1)**2)
    )

    ans[48] = (
        -g[48]
        + 2 * (-u[28] + u[48]) * (np.sqrt((-u[28] + u[48])**2 + (-u[29] + u[49] + 1)**2) - 1) / np.sqrt((-u[28] + u[48])**2 + (-u[29] + u[49] + 1)**2)
        + 2 * (u[48] - u[68]) * (np.sqrt((-u[48] + u[68])**2 + (-u[49] + u[69] + 1)**2) - 1) / np.sqrt((-u[48] + u[68])**2 + (-u[49] + u[69] + 1)**2)
        + 2 * (np.sqrt((-u[26] + u[48] + 1)**2 + (-u[27] + u[49] + 1)**2) - np.sqrt(2)) * (-u[26] + u[48] + 1) / np.sqrt((-u[26] + u[48] + 1)**2 + (-u[27] + u[49] + 1)**2)
        + 2 * (np.sqrt((u[30] - u[48] + 1)**2 + (-u[31] + u[49] + 1)**2) - np.sqrt(2)) * (-u[30] + u[48] - 1) / np.sqrt((u[30] - u[48] + 1)**2 + (-u[31] + u[49] + 1)**2)
        + 2 * (np.sqrt((-u[48] + u[70] + 1)**2 + (-u[49] + u[71] + 1)**2) - np.sqrt(2)) * (u[48] - u[70] - 1) / np.sqrt((-u[48] + u[70] + 1)**2 + (-u[49] + u[71] + 1)**2)
        + 2 * (np.sqrt((u[48] - u[66] + 1)**2 + (-u[49] + u[67] + 1)**2) - np.sqrt(2)) * (u[48] - u[66] + 1) / np.sqrt((u[48] - u[66] + 1)**2 + (-u[49] + u[67] + 1)**2)
        + 2 * (np.sqrt((-u[49] + u[51])**2 + (-u[48] + u[50] + 1)**2) - 1) * (u[48] - u[50] - 1) / np.sqrt((-u[49] + u[51])**2 + (-u[48] + u[50] + 1)**2)
        + 2 * (np.sqrt((-u[47] + u[49])**2 + (-u[46] + u[48] + 1)**2) - 1) * (-u[46] + u[48] + 1) / np.sqrt((-u[47] + u[49])**2 + (-u[46] + u[48] + 1)**2)
    )

    ans[49] = (
        -g[49]
        + 2 * (-u[47] + u[49]) * (np.sqrt((-u[47] + u[49])**2 + (-u[46] + u[48] + 1)**2) - 1) / np.sqrt((-u[47] + u[49])**2 + (-u[46] + u[48] + 1)**2)
        + 2 * (u[49] - u[51]) * (np.sqrt((-u[49] + u[51])**2 + (-u[48] + u[50] + 1)**2) - 1) / np.sqrt((-u[49] + u[51])**2 + (-u[48] + u[50] + 1)**2)
        + 2 * (np.sqrt((-u[26] + u[48] + 1)**2 + (-u[27] + u[49] + 1)**2) - np.sqrt(2)) * (-u[27] + u[49] + 1) / np.sqrt((-u[26] + u[48] + 1)**2 + (-u[27] + u[49] + 1)**2)
        + 2 * (np.sqrt((u[30] - u[48] + 1)**2 + (-u[31] + u[49] + 1)**2) - np.sqrt(2)) * (-u[31] + u[49] + 1) / np.sqrt((u[30] - u[48] + 1)**2 + (-u[31] + u[49] + 1)**2)
        + 2 * (np.sqrt((-u[48] + u[70] + 1)**2 + (-u[49] + u[71] + 1)**2) - np.sqrt(2)) * (u[49] - u[71] - 1) / np.sqrt((-u[48] + u[70] + 1)**2 + (-u[49] + u[71] + 1)**2)
        + 2 * (np.sqrt((u[48] - u[66] + 1)**2 + (-u[49] + u[67] + 1)**2) - np.sqrt(2)) * (u[49] - u[67] - 1) / np.sqrt((u[48] - u[66] + 1)**2 + (-u[49] + u[67] + 1)**2)
        + 2 * (np.sqrt((-u[48] + u[68])**2 + (-u[49] + u[69] + 1)**2) - 1) * (u[49] - u[69] - 1) / np.sqrt((-u[48] + u[68])**2 + (-u[49] + u[69] + 1)**2)
        + 2 * (np.sqrt((-u[28] + u[48])**2 + (-u[29] + u[49] + 1)**2) - 1) * (-u[29] + u[49] + 1) / np.sqrt((-u[28] + u[48])**2 + (-u[29] + u[49] + 1)**2)
    )

    ans[50] = (
        -g[50]
        + 2 * (-u[30] + u[50]) * (np.sqrt((-u[30] + u[50])**2 + (-u[31] + u[51] + 1)**2) - 1) / np.sqrt((-u[30] + u[50])**2 + (-u[31] + u[51] + 1)**2)
        + 2 * (u[50] - u[70]) * (np.sqrt((-u[50] + u[70])**2 + (-u[51] + u[71] + 1)**2) - 1) / np.sqrt((-u[50] + u[70])**2 + (-u[51] + u[71] + 1)**2)
        + 2 * (np.sqrt((-u[28] + u[50] + 1)**2 + (-u[29] + u[51] + 1)**2) - np.sqrt(2)) * (-u[28] + u[50] + 1) / np.sqrt((-u[28] + u[50] + 1)**2 + (-u[29] + u[51] + 1)**2)
        + 2 * (np.sqrt((u[32] - u[50] + 1)**2 + (-u[33] + u[51] + 1)**2) - np.sqrt(2)) * (-u[32] + u[50] - 1) / np.sqrt((u[32] - u[50] + 1)**2 + (-u[33] + u[51] + 1)**2)
        + 2 * (np.sqrt((-u[50] + u[72] + 1)**2 + (-u[51] + u[73] + 1)**2) - np.sqrt(2)) * (u[50] - u[72] - 1) / np.sqrt((-u[50] + u[72] + 1)**2 + (-u[51] + u[73] + 1)**2)
        + 2 * (np.sqrt((u[50] - u[68] + 1)**2 + (-u[51] + u[69] + 1)**2) - np.sqrt(2)) * (u[50] - u[68] + 1) / np.sqrt((u[50] - u[68] + 1)**2 + (-u[51] + u[69] + 1)**2)
        + 2 * (np.sqrt((-u[51] + u[53])**2 + (-u[50] + u[52] + 1)**2) - 1) * (u[50] - u[52] - 1) / np.sqrt((-u[51] + u[53])**2 + (-u[50] + u[52] + 1)**2)
        + 2 * (np.sqrt((-u[49] + u[51])**2 + (-u[48] + u[50] + 1)**2) - 1) * (-u[48] + u[50] + 1) / np.sqrt((-u[49] + u[51])**2 + (-u[48] + u[50] + 1)**2)
    )

    ans[51] = (
        -g[51]
        + 2 * (-u[49] + u[51]) * (np.sqrt((-u[49] + u[51])**2 + (-u[48] + u[50] + 1)**2) - 1) / np.sqrt((-u[49] + u[51])**2 + (-u[48] + u[50] + 1)**2)
        + 2 * (u[51] - u[53]) * (np.sqrt((-u[51] + u[53])**2 + (-u[50] + u[52] + 1)**2) - 1) / np.sqrt((-u[51] + u[53])**2 + (-u[50] + u[52] + 1)**2)
        + 2 * (np.sqrt((-u[28] + u[50] + 1)**2 + (-u[29] + u[51] + 1)**2) - np.sqrt(2)) * (-u[29] + u[51] + 1) / np.sqrt((-u[28] + u[50] + 1)**2 + (-u[29] + u[51] + 1)**2)
        + 2 * (np.sqrt((u[32] - u[50] + 1)**2 + (-u[33] + u[51] + 1)**2) - np.sqrt(2)) * (-u[33] + u[51] + 1) / np.sqrt((u[32] - u[50] + 1)**2 + (-u[33] + u[51] + 1)**2)
        + 2 * (np.sqrt((-u[50] + u[72] + 1)**2 + (-u[51] + u[73] + 1)**2) - np.sqrt(2)) * (u[51] - u[73] - 1) / np.sqrt((-u[50] + u[72] + 1)**2 + (-u[51] + u[73] + 1)**2)
        + 2 * (np.sqrt((u[50] - u[68] + 1)**2 + (-u[51] + u[69] + 1)**2) - np.sqrt(2)) * (u[51] - u[69] - 1) / np.sqrt((u[50] - u[68] + 1)**2 + (-u[51] + u[69] + 1)**2)
        + 2 * (np.sqrt((-u[50] + u[70])**2 + (-u[51] + u[71] + 1)**2) - 1) * (u[51] - u[71] - 1) / np.sqrt((-u[50] + u[70])**2 + (-u[51] + u[71] + 1)**2)
        + 2 * (np.sqrt((-u[30] + u[50])**2 + (-u[31] + u[51] + 1)**2) - 1) * (-u[31] + u[51] + 1) / np.sqrt((-u[30] + u[50])**2 + (-u[31] + u[51] + 1)**2)
    )

    ans[52] = (
        -g[52]
        + 2 * (-u[32] + u[52]) * (np.sqrt((-u[32] + u[52])**2 + (-u[33] + u[53] + 1)**2) - 1) / np.sqrt((-u[32] + u[52])**2 + (-u[33] + u[53] + 1)**2)
        + 2 * (u[52] - u[72]) * (np.sqrt((-u[52] + u[72])**2 + (-u[53] + u[73] + 1)**2) - 1) / np.sqrt((-u[52] + u[72])**2 + (-u[53] + u[73] + 1)**2)
        + 2 * (np.sqrt((-u[30] + u[52] + 1)**2 + (-u[31] + u[53] + 1)**2) - np.sqrt(2)) * (-u[30] + u[52] + 1) / np.sqrt((-u[30] + u[52] + 1)**2 + (-u[31] + u[53] + 1)**2)
        + 2 * (np.sqrt((u[34] - u[52] + 1)**2 + (-u[35] + u[53] + 1)**2) - np.sqrt(2)) * (-u[34] + u[52] - 1) / np.sqrt((u[34] - u[52] + 1)**2 + (-u[35] + u[53] + 1)**2)
        + 2 * (np.sqrt((-u[52] + u[74] + 1)**2 + (-u[53] + u[75] + 1)**2) - np.sqrt(2)) * (u[52] - u[74] - 1) / np.sqrt((-u[52] + u[74] + 1)**2 + (-u[53] + u[75] + 1)**2)
        + 2 * (np.sqrt((u[52] - u[70] + 1)**2 + (-u[53] + u[71] + 1)**2) - np.sqrt(2)) * (u[52] - u[70] + 1) / np.sqrt((u[52] - u[70] + 1)**2 + (-u[53] + u[71] + 1)**2)
        + 2 * (np.sqrt((-u[53] + u[55])**2 + (-u[52] + u[54] + 1)**2) - 1) * (u[52] - u[54] - 1) / np.sqrt((-u[53] + u[55])**2 + (-u[52] + u[54] + 1)**2)
        + 2 * (np.sqrt((-u[51] + u[53])**2 + (-u[50] + u[52] + 1)**2) - 1) * (-u[50] + u[52] + 1) / np.sqrt((-u[51] + u[53])**2 + (-u[50] + u[52] + 1)**2)
    )

    ans[53] = (
        -g[53]
        + 2 * (-u[51] + u[53]) * (np.sqrt((-u[51] + u[53])**2 + (-u[50] + u[52] + 1)**2) - 1) / np.sqrt((-u[51] + u[53])**2 + (-u[50] + u[52] + 1)**2)
        + 2 * (u[53] - u[55]) * (np.sqrt((-u[53] + u[55])**2 + (-u[52] + u[54] + 1)**2) - 1) / np.sqrt((-u[53] + u[55])**2 + (-u[52] + u[54] + 1)**2)
        + 2 * (np.sqrt((-u[30] + u[52] + 1)**2 + (-u[31] + u[53] + 1)**2) - np.sqrt(2)) * (-u[31] + u[53] + 1) / np.sqrt((-u[30] + u[52] + 1)**2 + (-u[31] + u[53] + 1)**2)
        + 2 * (np.sqrt((u[34] - u[52] + 1)**2 + (-u[35] + u[53] + 1)**2) - np.sqrt(2)) * (-u[35] + u[53] + 1) / np.sqrt((u[34] - u[52] + 1)**2 + (-u[35] + u[53] + 1)**2)
        + 2 * (np.sqrt((-u[52] + u[74] + 1)**2 + (-u[53] + u[75] + 1)**2) - np.sqrt(2)) * (u[53] - u[75] - 1) / np.sqrt((-u[52] + u[74] + 1)**2 + (-u[53] + u[75] + 1)**2)
        + 2 * (np.sqrt((u[52] - u[70] + 1)**2 + (-u[53] + u[71] + 1)**2) - np.sqrt(2)) * (u[53] - u[71] - 1) / np.sqrt((u[52] - u[70] + 1)**2 + (-u[53] + u[71] + 1)**2)
        + 2 * (np.sqrt((-u[52] + u[72])**2 + (-u[53] + u[73] + 1)**2) - 1) * (u[53] - u[73] - 1) / np.sqrt((-u[52] + u[72])**2 + (-u[53] + u[73] + 1)**2)
        + 2 * (np.sqrt((-u[32] + u[52])**2 + (-u[33] + u[53] + 1)**2) - 1) * (-u[33] + u[53] + 1) / np.sqrt((-u[32] + u[52])**2 + (-u[33] + u[53] + 1)**2)
    )

    ans[54] = (
        -g[54]
        + 2 * (-u[34] + u[54]) * (np.sqrt((-u[34] + u[54])**2 + (-u[35] + u[55] + 1)**2) - 1) / np.sqrt((-u[34] + u[54])**2 + (-u[35] + u[55] + 1)**2)
        + 2 * (u[54] - u[74]) * (np.sqrt((-u[54] + u[74])**2 + (-u[55] + u[75] + 1)**2) - 1) / np.sqrt((-u[54] + u[74])**2 + (-u[55] + u[75] + 1)**2)
        + 2 * (np.sqrt((-u[32] + u[54] + 1)**2 + (-u[33] + u[55] + 1)**2) - np.sqrt(2)) * (-u[32] + u[54] + 1) / np.sqrt((-u[32] + u[54] + 1)**2 + (-u[33] + u[55] + 1)**2)
        + 2 * (np.sqrt((u[36] - u[54] + 1)**2 + (-u[37] + u[55] + 1)**2) - np.sqrt(2)) * (-u[36] + u[54] - 1) / np.sqrt((u[36] - u[54] + 1)**2 + (-u[37] + u[55] + 1)**2)
        + 2 * (np.sqrt((-u[54] + u[76] + 1)**2 + (-u[55] + u[77] + 1)**2) - np.sqrt(2)) * (u[54] - u[76] - 1) / np.sqrt((-u[54] + u[76] + 1)**2 + (-u[55] + u[77] + 1)**2)
        + 2 * (np.sqrt((u[54] - u[72] + 1)**2 + (-u[55] + u[73] + 1)**2) - np.sqrt(2)) * (u[54] - u[72] + 1) / np.sqrt((u[54] - u[72] + 1)**2 + (-u[55] + u[73] + 1)**2)
        + 2 * (np.sqrt((-u[55] + u[57])**2 + (-u[54] + u[56] + 1)**2) - 1) * (u[54] - u[56] - 1) / np.sqrt((-u[55] + u[57])**2 + (-u[54] + u[56] + 1)**2)
        + 2 * (np.sqrt((-u[53] + u[55])**2 + (-u[52] + u[54] + 1)**2) - 1) * (-u[52] + u[54] + 1) / np.sqrt((-u[53] + u[55])**2 + (-u[52] + u[54] + 1)**2)
    )

    ans[55] = (
        -g[55]
        + 2 * (-u[53] + u[55]) * (np.sqrt((-u[53] + u[55])**2 + (-u[52] + u[54] + 1)**2) - 1) / np.sqrt((-u[53] + u[55])**2 + (-u[52] + u[54] + 1)**2)
        + 2 * (u[55] - u[57]) * (np.sqrt((-u[55] + u[57])**2 + (-u[54] + u[56] + 1)**2) - 1) / np.sqrt((-u[55] + u[57])**2 + (-u[54] + u[56] + 1)**2)
        + 2 * (np.sqrt((-u[32] + u[54] + 1)**2 + (-u[33] + u[55] + 1)**2) - np.sqrt(2)) * (-u[33] + u[55] + 1) / np.sqrt((-u[32] + u[54] + 1)**2 + (-u[33] + u[55] + 1)**2)
        + 2 * (np.sqrt((u[36] - u[54] + 1)**2 + (-u[37] + u[55] + 1)**2) - np.sqrt(2)) * (-u[37] + u[55] + 1) / np.sqrt((u[36] - u[54] + 1)**2 + (-u[37] + u[55] + 1)**2)
        + 2 * (np.sqrt((-u[54] + u[76] + 1)**2 + (-u[55] + u[77] + 1)**2) - np.sqrt(2)) * (u[55] - u[77] - 1) / np.sqrt((-u[54] + u[76] + 1)**2 + (-u[55] + u[77] + 1)**2)
        + 2 * (np.sqrt((u[54] - u[72] + 1)**2 + (-u[55] + u[73] + 1)**2) - np.sqrt(2)) * (u[55] - u[73] - 1) / np.sqrt((u[54] - u[72] + 1)**2 + (-u[55] + u[73] + 1)**2)
        + 2 * (np.sqrt((-u[54] + u[74])**2 + (-u[55] + u[75] + 1)**2) - 1) * (u[55] - u[75] - 1) / np.sqrt((-u[54] + u[74])**2 + (-u[55] + u[75] + 1)**2)
        + 2 * (np.sqrt((-u[34] + u[54])**2 + (-u[35] + u[55] + 1)**2) - 1) * (-u[35] + u[55] + 1) / np.sqrt((-u[34] + u[54])**2 + (-u[35] + u[55] + 1)**2)
    )

    ans[56] = (
        -g[56]
        + 2 * (-u[36] + u[56]) * (np.sqrt((-u[36] + u[56])**2 + (-u[37] + u[57] + 1)**2) - 1) / np.sqrt((-u[36] + u[56])**2 + (-u[37] + u[57] + 1)**2)
        + 2 * (u[56] - u[76]) * (np.sqrt((-u[56] + u[76])**2 + (-u[57] + u[77] + 1)**2) - 1) / np.sqrt((-u[56] + u[76])**2 + (-u[57] + u[77] + 1)**2)
        + 2 * (np.sqrt((-u[34] + u[56] + 1)**2 + (-u[35] + u[57] + 1)**2) - np.sqrt(2)) * (-u[34] + u[56] + 1) / np.sqrt((-u[34] + u[56] + 1)**2 + (-u[35] + u[57] + 1)**2)
        + 2 * (np.sqrt((u[38] - u[56] + 1)**2 + (-u[39] + u[57] + 1)**2) - np.sqrt(2)) * (-u[38] + u[56] - 1) / np.sqrt((u[38] - u[56] + 1)**2 + (-u[39] + u[57] + 1)**2)
        + 2 * (np.sqrt((-u[56] + u[78] + 1)**2 + (-u[57] + u[79] + 1)**2) - np.sqrt(2)) * (u[56] - u[78] - 1) / np.sqrt((-u[56] + u[78] + 1)**2 + (-u[57] + u[79] + 1)**2)
        + 2 * (np.sqrt((u[56] - u[74] + 1)**2 + (-u[57] + u[75] + 1)**2) - np.sqrt(2)) * (u[56] - u[74] + 1) / np.sqrt((u[56] - u[74] + 1)**2 + (-u[57] + u[75] + 1)**2)
        + 2 * (np.sqrt((-u[57] + u[59])**2 + (-u[56] + u[58] + 1)**2) - 1) * (u[56] - u[58] - 1) / np.sqrt((-u[57] + u[59])**2 + (-u[56] + u[58] + 1)**2)
        + 2 * (np.sqrt((-u[55] + u[57])**2 + (-u[54] + u[56] + 1)**2) - 1) * (-u[54] + u[56] + 1) / np.sqrt((-u[55] + u[57])**2 + (-u[54] + u[56] + 1)**2)
    )

    ans[57] = (
        -g[57]
        + 2 * (-u[55] + u[57]) * (np.sqrt((-u[55] + u[57])**2 + (-u[54] + u[56] + 1)**2) - 1) / np.sqrt((-u[55] + u[57])**2 + (-u[54] + u[56] + 1)**2)
        + 2 * (u[57] - u[59]) * (np.sqrt((-u[57] + u[59])**2 + (-u[56] + u[58] + 1)**2) - 1) / np.sqrt((-u[57] + u[59])**2 + (-u[56] + u[58] + 1)**2)
        + 2 * (np.sqrt((-u[34] + u[56] + 1)**2 + (-u[35] + u[57] + 1)**2) - np.sqrt(2)) * (-u[35] + u[57] + 1) / np.sqrt((-u[34] + u[56] + 1)**2 + (-u[35] + u[57] + 1)**2)
        + 2 * (np.sqrt((u[38] - u[56] + 1)**2 + (-u[39] + u[57] + 1)**2) - np.sqrt(2)) * (-u[39] + u[57] + 1) / np.sqrt((u[38] - u[56] + 1)**2 + (-u[39] + u[57] + 1)**2)
        + 2 * (np.sqrt((-u[56] + u[78] + 1)**2 + (-u[57] + u[79] + 1)**2) - np.sqrt(2)) * (u[57] - u[79] - 1) / np.sqrt((-u[56] + u[78] + 1)**2 + (-u[57] + u[79] + 1)**2)
        + 2 * (np.sqrt((u[56] - u[74] + 1)**2 + (-u[57] + u[75] + 1)**2) - np.sqrt(2)) * (u[57] - u[75] - 1) / np.sqrt((u[56] - u[74] + 1)**2 + (-u[57] + u[75] + 1)**2)
        + 2 * (np.sqrt((-u[56] + u[76])**2 + (-u[57] + u[77] + 1)**2) - 1) * (u[57] - u[77] - 1) / np.sqrt((-u[56] + u[76])**2 + (-u[57] + u[77] + 1)**2)
        + 2 * (np.sqrt((-u[36] + u[56])**2 + (-u[37] + u[57] + 1)**2) - 1) * (-u[37] + u[57] + 1) / np.sqrt((-u[36] + u[56])**2 + (-u[37] + u[57] + 1)**2)
    )

    ans[58] = (
        -g[58]
        + 2 * (-u[38] + u[58]) * (np.sqrt((-u[38] + u[58])**2 + (-u[39] + u[59] + 1)**2) - 1) / np.sqrt((-u[38] + u[58])**2 + (-u[39] + u[59] + 1)**2)
        + 2 * (u[58] - u[78]) * (np.sqrt((-u[58] + u[78])**2 + (-u[59] + u[79] + 1)**2) - 1) / np.sqrt((-u[58] + u[78])**2 + (-u[59] + u[79] + 1)**2)
        + 2 * (np.sqrt((-u[36] + u[58] + 1)**2 + (-u[37] + u[59] + 1)**2) - np.sqrt(2)) * (-u[36] + u[58] + 1) / np.sqrt((-u[36] + u[58] + 1)**2 + (-u[37] + u[59] + 1)**2)
        + 2 * (np.sqrt((u[58] - u[76] + 1)**2 + (-u[59] + u[77] + 1)**2) - np.sqrt(2)) * (u[58] - u[76] + 1) / np.sqrt((u[58] - u[76] + 1)**2 + (-u[59] + u[77] + 1)**2)
        + 2 * (np.sqrt((-u[57] + u[59])**2 + (-u[56] + u[58] + 1)**2) - 1) * (-u[56] + u[58] + 1) / np.sqrt((-u[57] + u[59])**2 + (-u[56] + u[58] + 1)**2)
    )

    ans[59] = (
        -g[59]
        + 2 * (-u[57] + u[59]) * (np.sqrt((-u[57] + u[59])**2 + (-u[56] + u[58] + 1)**2) - 1) / np.sqrt((-u[57] + u[59])**2 + (-u[56] + u[58] + 1)**2)
        + 2 * (np.sqrt((-u[36] + u[58] + 1)**2 + (-u[37] + u[59] + 1)**2) - np.sqrt(2)) * (-u[37] + u[59] + 1) / np.sqrt((-u[36] + u[58] + 1)**2 + (-u[37] + u[59] + 1)**2)
        + 2 * (np.sqrt((u[58] - u[76] + 1)**2 + (-u[59] + u[77] + 1)**2) - np.sqrt(2)) * (u[59] - u[77] - 1) / np.sqrt((u[58] - u[76] + 1)**2 + (-u[59] + u[77] + 1)**2)
        + 2 * (np.sqrt((-u[58] + u[78])**2 + (-u[59] + u[79] + 1)**2) - 1) * (u[59] - u[79] - 1) / np.sqrt((-u[58] + u[78])**2 + (-u[59] + u[79] + 1)**2)
        + 2 * (np.sqrt((-u[38] + u[58])**2 + (-u[39] + u[59] + 1)**2) - 1) * (-u[39] + u[59] + 1) / np.sqrt((-u[38] + u[58])**2 + (-u[39] + u[59] + 1)**2)
    )

    ans[60] = (
        -g[60]
        + 2 * (-u[40] + u[60]) * (np.sqrt((-u[40] + u[60])**2 + (-u[41] + u[61] + 1)**2) - 1) / np.sqrt((-u[40] + u[60])**2 + (-u[41] + u[61] + 1)**2)
        + 2 * (np.sqrt((u[42] - u[60] + 1)**2 + (-u[43] + u[61] + 1)**2) - np.sqrt(2)) * (-u[42] + u[60] - 1) / np.sqrt((u[42] - u[60] + 1)**2 + (-u[43] + u[61] + 1)**2)
        + 2 * (np.sqrt((-u[61] + u[63])**2 + (-u[60] + u[62] + 1)**2) - 1) * (u[60] - u[62] - 1) / np.sqrt((-u[61] + u[63])**2 + (-u[60] + u[62] + 1)**2)
    )

    ans[61] = (
        -g[61]
        + 2 * (u[61] - u[63]) * (np.sqrt((-u[61] + u[63])**2 + (-u[60] + u[62] + 1)**2) - 1) / np.sqrt((-u[61] + u[63])**2 + (-u[60] + u[62] + 1)**2)
        + 2 * (np.sqrt((u[42] - u[60] + 1)**2 + (-u[43] + u[61] + 1)**2) - np.sqrt(2)) * (-u[43] + u[61] + 1) / np.sqrt((u[42] - u[60] + 1)**2 + (-u[43] + u[61] + 1)**2)
        + 2 * (np.sqrt((-u[40] + u[60])**2 + (-u[41] + u[61] + 1)**2) - 1) * (-u[41] + u[61] + 1) / np.sqrt((-u[40] + u[60])**2 + (-u[41] + u[61] + 1)**2)
    )

    ans[62] = (
        -g[62]
        + 2 * (-u[42] + u[62]) * (np.sqrt((-u[42] + u[62])**2 + (-u[43] + u[63] + 1)**2) - 1) / np.sqrt((-u[42] + u[62])**2 + (-u[43] + u[63] + 1)**2)
        + 2 * (np.sqrt((-u[40] + u[62] + 1)**2 + (-u[41] + u[63] + 1)**2) - np.sqrt(2)) * (-u[40] + u[62] + 1) / np.sqrt((-u[40] + u[62] + 1)**2 + (-u[41] + u[63] + 1)**2)
        + 2 * (np.sqrt((u[44] - u[62] + 1)**2 + (-u[45] + u[63] + 1)**2) - np.sqrt(2)) * (-u[44] + u[62] - 1) / np.sqrt((u[44] - u[62] + 1)**2 + (-u[45] + u[63] + 1)**2)
        + 2 * (np.sqrt((-u[63] + u[65])**2 + (-u[62] + u[64] + 1)**2) - 1) * (u[62] - u[64] - 1) / np.sqrt((-u[63] + u[65])**2 + (-u[62] + u[64] + 1)**2)
        + 2 * (np.sqrt((-u[61] + u[63])**2 + (-u[60] + u[62] + 1)**2) - 1) * (-u[60] + u[62] + 1) / np.sqrt((-u[61] + u[63])**2 + (-u[60] + u[62] + 1)**2)
    )

    ans[63] = (
        -g[63]
        + 2 * (-u[61] + u[63]) * (np.sqrt((-u[61] + u[63])**2 + (-u[60] + u[62] + 1)**2) - 1) / np.sqrt((-u[61] + u[63])**2 + (-u[60] + u[62] + 1)**2)
        + 2 * (u[63] - u[65]) * (np.sqrt((-u[63] + u[65])**2 + (-u[62] + u[64] + 1)**2) - 1) / np.sqrt((-u[63] + u[65])**2 + (-u[62] + u[64] + 1)**2)
        + 2 * (np.sqrt((-u[40] + u[62] + 1)**2 + (-u[41] + u[63] + 1)**2) - np.sqrt(2)) * (-u[41] + u[63] + 1) / np.sqrt((-u[40] + u[62] + 1)**2 + (-u[41] + u[63] + 1)**2)
        + 2 * (np.sqrt((u[44] - u[62] + 1)**2 + (-u[45] + u[63] + 1)**2) - np.sqrt(2)) * (-u[45] + u[63] + 1) / np.sqrt((u[44] - u[62] + 1)**2 + (-u[45] + u[63] + 1)**2)
        + 2 * (np.sqrt((-u[42] + u[62])**2 + (-u[43] + u[63] + 1)**2) - 1) * (-u[43] + u[63] + 1) / np.sqrt((-u[42] + u[62])**2 + (-u[43] + u[63] + 1)**2)
    )

    ans[64] = (
        -g[64]
        + 2 * (-u[44] + u[64]) * (np.sqrt((-u[44] + u[64])**2 + (-u[45] + u[65] + 1)**2) - 1) / np.sqrt((-u[44] + u[64])**2 + (-u[45] + u[65] + 1)**2)
        + 2 * (np.sqrt((-u[42] + u[64] + 1)**2 + (-u[43] + u[65] + 1)**2) - np.sqrt(2)) * (-u[42] + u[64] + 1) / np.sqrt((-u[42] + u[64] + 1)**2 + (-u[43] + u[65] + 1)**2)
        + 2 * (np.sqrt((u[46] - u[64] + 1)**2 + (-u[47] + u[65] + 1)**2) - np.sqrt(2)) * (-u[46] + u[64] - 1) / np.sqrt((u[46] - u[64] + 1)**2 + (-u[47] + u[65] + 1)**2)
        + 2 * (np.sqrt((-u[65] + u[67])**2 + (-u[64] + u[66] + 1)**2) - 1) * (u[64] - u[66] - 1) / np.sqrt((-u[65] + u[67])**2 + (-u[64] + u[66] + 1)**2)
        + 2 * (np.sqrt((-u[63] + u[65])**2 + (-u[62] + u[64] + 1)**2) - 1) * (-u[62] + u[64] + 1) / np.sqrt((-u[63] + u[65])**2 + (-u[62] + u[64] + 1)**2)
    )

    ans[65] = (
        -g[65]
        + 2 * (-u[63] + u[65]) * (np.sqrt((-u[63] + u[65])**2 + (-u[62] + u[64] + 1)**2) - 1) / np.sqrt((-u[63] + u[65])**2 + (-u[62] + u[64] + 1)**2)
        + 2 * (u[65] - u[67]) * (np.sqrt((-u[65] + u[67])**2 + (-u[64] + u[66] + 1)**2) - 1) / np.sqrt((-u[65] + u[67])**2 + (-u[64] + u[66] + 1)**2)
        + 2 * (np.sqrt((-u[42] + u[64] + 1)**2 + (-u[43] + u[65] + 1)**2) - np.sqrt(2)) * (-u[43] + u[65] + 1) / np.sqrt((-u[42] + u[64] + 1)**2 + (-u[43] + u[65] + 1)**2)
        + 2 * (np.sqrt((u[46] - u[64] + 1)**2 + (-u[47] + u[65] + 1)**2) - np.sqrt(2)) * (-u[47] + u[65] + 1) / np.sqrt((u[46] - u[64] + 1)**2 + (-u[47] + u[65] + 1)**2)
        + 2 * (np.sqrt((-u[44] + u[64])**2 + (-u[45] + u[65] + 1)**2) - 1) * (-u[45] + u[65] + 1) / np.sqrt((-u[44] + u[64])**2 + (-u[45] + u[65] + 1)**2)
    )

    ans[66] = (
        -g[66]
        + 2 * (-u[46] + u[66]) * (np.sqrt((-u[46] + u[66])**2 + (-u[47] + u[67] + 1)**2) - 1) / np.sqrt((-u[46] + u[66])**2 + (-u[47] + u[67] + 1)**2)
        + 2 * (np.sqrt((-u[44] + u[66] + 1)**2 + (-u[45] + u[67] + 1)**2) - np.sqrt(2)) * (-u[44] + u[66] + 1) / np.sqrt((-u[44] + u[66] + 1)**2 + (-u[45] + u[67] + 1)**2)
        + 2 * (np.sqrt((u[48] - u[66] + 1)**2 + (-u[49] + u[67] + 1)**2) - np.sqrt(2)) * (-u[48] + u[66] - 1) / np.sqrt((u[48] - u[66] + 1)**2 + (-u[49] + u[67] + 1)**2)
        + 2 * (np.sqrt((-u[67] + u[69])**2 + (-u[66] + u[68] + 1)**2) - 1) * (u[66] - u[68] - 1) / np.sqrt((-u[67] + u[69])**2 + (-u[66] + u[68] + 1)**2)
        + 2 * (np.sqrt((-u[65] + u[67])**2 + (-u[64] + u[66] + 1)**2) - 1) * (-u[64] + u[66] + 1) / np.sqrt((-u[65] + u[67])**2 + (-u[64] + u[66] + 1)**2)
    )

    ans[67] = (
        -g[67]
        + 2 * (-u[65] + u[67]) * (np.sqrt((-u[65] + u[67])**2 + (-u[64] + u[66] + 1)**2) - 1) / np.sqrt((-u[65] + u[67])**2 + (-u[64] + u[66] + 1)**2)
        + 2 * (u[67] - u[69]) * (np.sqrt((-u[67] + u[69])**2 + (-u[66] + u[68] + 1)**2) - 1) / np.sqrt((-u[67] + u[69])**2 + (-u[66] + u[68] + 1)**2)
        + 2 * (np.sqrt((-u[44] + u[66] + 1)**2 + (-u[45] + u[67] + 1)**2) - np.sqrt(2)) * (-u[45] + u[67] + 1) / np.sqrt((-u[44] + u[66] + 1)**2 + (-u[45] + u[67] + 1)**2)
        + 2 * (np.sqrt((u[48] - u[66] + 1)**2 + (-u[49] + u[67] + 1)**2) - np.sqrt(2)) * (-u[49] + u[67] + 1) / np.sqrt((u[48] - u[66] + 1)**2 + (-u[49] + u[67] + 1)**2)
        + 2 * (np.sqrt((-u[46] + u[66])**2 + (-u[47] + u[67] + 1)**2) - 1) * (-u[47] + u[67] + 1) / np.sqrt((-u[46] + u[66])**2 + (-u[47] + u[67] + 1)**2)
    )

    ans[68] = (
        -g[68]
        + 2 * (-u[48] + u[68]) * (np.sqrt((-u[48] + u[68])**2 + (-u[49] + u[69] + 1)**2) - 1) / np.sqrt((-u[48] + u[68])**2 + (-u[49] + u[69] + 1)**2)
        + 2 * (np.sqrt((-u[46] + u[68] + 1)**2 + (-u[47] + u[69] + 1)**2) - np.sqrt(2)) * (-u[46] + u[68] + 1) / np.sqrt((-u[46] + u[68] + 1)**2 + (-u[47] + u[69] + 1)**2)
        + 2 * (np.sqrt((u[50] - u[68] + 1)**2 + (-u[51] + u[69] + 1)**2) - np.sqrt(2)) * (-u[50] + u[68] - 1) / np.sqrt((u[50] - u[68] + 1)**2 + (-u[51] + u[69] + 1)**2)
        + 2 * (np.sqrt((-u[69] + u[71])**2 + (-u[68] + u[70] + 1)**2) - 1) * (u[68] - u[70] - 1) / np.sqrt((-u[69] + u[71])**2 + (-u[68] + u[70] + 1)**2)
        + 2 * (np.sqrt((-u[67] + u[69])**2 + (-u[66] + u[68] + 1)**2) - 1) * (-u[66] + u[68] + 1) / np.sqrt((-u[67] + u[69])**2 + (-u[66] + u[68] + 1)**2)
    )

    ans[69] = (
        -g[69]
        + 2 * (-u[67] + u[69]) * (np.sqrt((-u[67] + u[69])**2 + (-u[66] + u[68] + 1)**2) - 1) / np.sqrt((-u[67] + u[69])**2 + (-u[66] + u[68] + 1)**2)
        + 2 * (u[69] - u[71]) * (np.sqrt((-u[69] + u[71])**2 + (-u[68] + u[70] + 1)**2) - 1) / np.sqrt((-u[69] + u[71])**2 + (-u[68] + u[70] + 1)**2)
        + 2 * (np.sqrt((-u[46] + u[68] + 1)**2 + (-u[47] + u[69] + 1)**2) - np.sqrt(2)) * (-u[47] + u[69] + 1) / np.sqrt((-u[46] + u[68] + 1)**2 + (-u[47] + u[69] + 1)**2)
        + 2 * (np.sqrt((u[50] - u[68] + 1)**2 + (-u[51] + u[69] + 1)**2) - np.sqrt(2)) * (-u[51] + u[69] + 1) / np.sqrt((u[50] - u[68] + 1)**2 + (-u[51] + u[69] + 1)**2)
        + 2 * (np.sqrt((-u[48] + u[68])**2 + (-u[49] + u[69] + 1)**2) - 1) * (-u[49] + u[69] + 1) / np.sqrt((-u[48] + u[68])**2 + (-u[49] + u[69] + 1)**2)
    )

    ans[70] = (
        -g[70]
        + 2 * (-u[50] + u[70]) * (np.sqrt((-u[50] + u[70])**2 + (-u[51] + u[71] + 1)**2) - 1) / np.sqrt((-u[50] + u[70])**2 + (-u[51] + u[71] + 1)**2)
        + 2 * (np.sqrt((-u[48] + u[70] + 1)**2 + (-u[49] + u[71] + 1)**2) - np.sqrt(2)) * (-u[48] + u[70] + 1) / np.sqrt((-u[48] + u[70] + 1)**2 + (-u[49] + u[71] + 1)**2)
        + 2 * (np.sqrt((u[52] - u[70] + 1)**2 + (-u[53] + u[71] + 1)**2) - np.sqrt(2)) * (-u[52] + u[70] - 1) / np.sqrt((u[52] - u[70] + 1)**2 + (-u[53] + u[71] + 1)**2)
        + 2 * (np.sqrt((-u[71] + u[73])**2 + (-u[70] + u[72] + 1)**2) - 1) * (u[70] - u[72] - 1) / np.sqrt((-u[71] + u[73])**2 + (-u[70] + u[72] + 1)**2)
        + 2 * (np.sqrt((-u[69] + u[71])**2 + (-u[68] + u[70] + 1)**2) - 1) * (-u[68] + u[70] + 1) / np.sqrt((-u[69] + u[71])**2 + (-u[68] + u[70] + 1)**2)
    )

    ans[71] = (
        -g[71]
        + 2 * (-u[69] + u[71]) * (np.sqrt((-u[69] + u[71])**2 + (-u[68] + u[70] + 1)**2) - 1) / np.sqrt((-u[69] + u[71])**2 + (-u[68] + u[70] + 1)**2)
        + 2 * (u[71] - u[73]) * (np.sqrt((-u[71] + u[73])**2 + (-u[70] + u[72] + 1)**2) - 1) / np.sqrt((-u[71] + u[73])**2 + (-u[70] + u[72] + 1)**2)
        + 2 * (np.sqrt((-u[48] + u[70] + 1)**2 + (-u[49] + u[71] + 1)**2) - np.sqrt(2)) * (-u[49] + u[71] + 1) / np.sqrt((-u[48] + u[70] + 1)**2 + (-u[49] + u[71] + 1)**2)
        + 2 * (np.sqrt((u[52] - u[70] + 1)**2 + (-u[53] + u[71] + 1)**2) - np.sqrt(2)) * (-u[53] + u[71] + 1) / np.sqrt((u[52] - u[70] + 1)**2 + (-u[53] + u[71] + 1)**2)
        + 2 * (np.sqrt((-u[50] + u[70])**2 + (-u[51] + u[71] + 1)**2) - 1) * (-u[51] + u[71] + 1) / np.sqrt((-u[50] + u[70])**2 + (-u[51] + u[71] + 1)**2)
    )

    ans[72] = (
        -g[72]
        + 2 * (-u[52] + u[72]) * (np.sqrt((-u[52] + u[72])**2 + (-u[53] + u[73] + 1)**2) - 1) / np.sqrt((-u[52] + u[72])**2 + (-u[53] + u[73] + 1)**2)
        + 2 * (np.sqrt((-u[50] + u[72] + 1)**2 + (-u[51] + u[73] + 1)**2) - np.sqrt(2)) * (-u[50] + u[72] + 1) / np.sqrt((-u[50] + u[72] + 1)**2 + (-u[51] + u[73] + 1)**2)
        + 2 * (np.sqrt((u[54] - u[72] + 1)**2 + (-u[55] + u[73] + 1)**2) - np.sqrt(2)) * (-u[54] + u[72] - 1) / np.sqrt((u[54] - u[72] + 1)**2 + (-u[55] + u[73] + 1)**2)
        + 2 * (np.sqrt((-u[73] + u[75])**2 + (-u[72] + u[74] + 1)**2) - 1) * (u[72] - u[74] - 1) / np.sqrt((-u[73] + u[75])**2 + (-u[72] + u[74] + 1)**2)
        + 2 * (np.sqrt((-u[71] + u[73])**2 + (-u[70] + u[72] + 1)**2) - 1) * (-u[70] + u[72] + 1) / np.sqrt((-u[71] + u[73])**2 + (-u[70] + u[72] + 1)**2)
    )

    ans[73] = (
        -g[73]
        + 2 * (-u[71] + u[73]) * (np.sqrt((-u[71] + u[73])**2 + (-u[70] + u[72] + 1)**2) - 1) / np.sqrt((-u[71] + u[73])**2 + (-u[70] + u[72] + 1)**2)
        + 2 * (u[73] - u[75]) * (np.sqrt((-u[73] + u[75])**2 + (-u[72] + u[74] + 1)**2) - 1) / np.sqrt((-u[73] + u[75])**2 + (-u[72] + u[74] + 1)**2)
        + 2 * (np.sqrt((-u[50] + u[72] + 1)**2 + (-u[51] + u[73] + 1)**2) - np.sqrt(2)) * (-u[51] + u[73] + 1) / np.sqrt((-u[50] + u[72] + 1)**2 + (-u[51] + u[73] + 1)**2)
        + 2 * (np.sqrt((u[54] - u[72] + 1)**2 + (-u[55] + u[73] + 1)**2) - np.sqrt(2)) * (-u[55] + u[73] + 1) / np.sqrt((u[54] - u[72] + 1)**2 + (-u[55] + u[73] + 1)**2)
        + 2 * (np.sqrt((-u[52] + u[72])**2 + (-u[53] + u[73] + 1)**2) - 1) * (-u[53] + u[73] + 1) / np.sqrt((-u[52] + u[72])**2 + (-u[53] + u[73] + 1)**2)
    )

    ans[74] = (
        -g[74]
        + 2 * (-u[54] + u[74]) * (np.sqrt((-u[54] + u[74])**2 + (-u[55] + u[75] + 1)**2) - 1) / np.sqrt((-u[54] + u[74])**2 + (-u[55] + u[75] + 1)**2)
        + 2 * (np.sqrt((-u[52] + u[74] + 1)**2 + (-u[53] + u[75] + 1)**2) - np.sqrt(2)) * (-u[52] + u[74] + 1) / np.sqrt((-u[52] + u[74] + 1)**2 + (-u[53] + u[75] + 1)**2)
        + 2 * (np.sqrt((u[56] - u[74] + 1)**2 + (-u[57] + u[75] + 1)**2) - np.sqrt(2)) * (-u[56] + u[74] - 1) / np.sqrt((u[56] - u[74] + 1)**2 + (-u[57] + u[75] + 1)**2)
        + 2 * (np.sqrt((-u[75] + u[77])**2 + (-u[74] + u[76] + 1)**2) - 1) * (u[74] - u[76] - 1) / np.sqrt((-u[75] + u[77])**2 + (-u[74] + u[76] + 1)**2)
        + 2 * (np.sqrt((-u[73] + u[75])**2 + (-u[72] + u[74] + 1)**2) - 1) * (-u[72] + u[74] + 1) / np.sqrt((-u[73] + u[75])**2 + (-u[72] + u[74] + 1)**2)
    )

    ans[75] = (
        -g[75]
        + 2 * (-u[73] + u[75]) * (np.sqrt((-u[73] + u[75])**2 + (-u[72] + u[74] + 1)**2) - 1) / np.sqrt((-u[73] + u[75])**2 + (-u[72] + u[74] + 1)**2)
        + 2 * (u[75] - u[77]) * (np.sqrt((-u[75] + u[77])**2 + (-u[74] + u[76] + 1)**2) - 1) / np.sqrt((-u[75] + u[77])**2 + (-u[74] + u[76] + 1)**2)
        + 2 * (np.sqrt((-u[52] + u[74] + 1)**2 + (-u[53] + u[75] + 1)**2) - np.sqrt(2)) * (-u[53] + u[75] + 1) / np.sqrt((-u[52] + u[74] + 1)**2 + (-u[53] + u[75] + 1)**2)
        + 2 * (np.sqrt((u[56] - u[74] + 1)**2 + (-u[57] + u[75] + 1)**2) - np.sqrt(2)) * (-u[57] + u[75] + 1) / np.sqrt((u[56] - u[74] + 1)**2 + (-u[57] + u[75] + 1)**2)
        + 2 * (np.sqrt((-u[54] + u[74])**2 + (-u[55] + u[75] + 1)**2) - 1) * (-u[55] + u[75] + 1) / np.sqrt((-u[54] + u[74])**2 + (-u[55] + u[75] + 1)**2)
    )

    ans[76] = (
        -g[76]
        + 2 * (-u[56] + u[76]) * (np.sqrt((-u[56] + u[76])**2 + (-u[57] + u[77] + 1)**2) - 1) / np.sqrt((-u[56] + u[76])**2 + (-u[57] + u[77] + 1)**2)
        + 2 * (np.sqrt((-u[54] + u[76] + 1)**2 + (-u[55] + u[77] + 1)**2) - np.sqrt(2)) * (-u[54] + u[76] + 1) / np.sqrt((-u[54] + u[76] + 1)**2 + (-u[55] + u[77] + 1)**2)
        + 2 * (np.sqrt((u[58] - u[76] + 1)**2 + (-u[59] + u[77] + 1)**2) - np.sqrt(2)) * (-u[58] + u[76] - 1) / np.sqrt((u[58] - u[76] + 1)**2 + (-u[59] + u[77] + 1)**2)
        + 2 * (np.sqrt((-u[77] + u[79])**2 + (-u[76] + u[78] + 1)**2) - 1) * (u[76] - u[78] - 1) / np.sqrt((-u[77] + u[79])**2 + (-u[76] + u[78] + 1)**2)
        + 2 * (np.sqrt((-u[75] + u[77])**2 + (-u[74] + u[76] + 1)**2) - 1) * (-u[74] + u[76] + 1) / np.sqrt((-u[75] + u[77])**2 + (-u[74] + u[76] + 1)**2)
    )

    ans[77] = (
        -g[77]
        + 2 * (-u[75] + u[77]) * (np.sqrt((-u[75] + u[77])**2 + (-u[74] + u[76] + 1)**2) - 1) / np.sqrt((-u[75] + u[77])**2 + (-u[74] + u[76] + 1)**2)
        + 2 * (u[77] - u[79]) * (np.sqrt((-u[77] + u[79])**2 + (-u[76] + u[78] + 1)**2) - 1) / np.sqrt((-u[77] + u[79])**2 + (-u[76] + u[78] + 1)**2)
        + 2 * (np.sqrt((-u[54] + u[76] + 1)**2 + (-u[55] + u[77] + 1)**2) - np.sqrt(2)) * (-u[55] + u[77] + 1) / np.sqrt((-u[54] + u[76] + 1)**2 + (-u[55] + u[77] + 1)**2)
        + 2 * (np.sqrt((u[58] - u[76] + 1)**2 + (-u[59] + u[77] + 1)**2) - np.sqrt(2)) * (-u[59] + u[77] + 1) / np.sqrt((u[58] - u[76] + 1)**2 + (-u[59] + u[77] + 1)**2)
        + 2 * (np.sqrt((-u[56] + u[76])**2 + (-u[57] + u[77] + 1)**2) - 1) * (-u[57] + u[77] + 1) / np.sqrt((-u[56] + u[76])**2 + (-u[57] + u[77] + 1)**2)
    )

    ans[78] = (
        -g[78]
        + 2 * (-u[58] + u[78]) * (np.sqrt((-u[58] + u[78])**2 + (-u[59] + u[79] + 1)**2) - 1) / np.sqrt((-u[58] + u[78])**2 + (-u[59] + u[79] + 1)**2)
        + 2 * (np.sqrt((-u[56] + u[78] + 1)**2 + (-u[57] + u[79] + 1)**2) - np.sqrt(2)) * (-u[56] + u[78] + 1) / np.sqrt((-u[56] + u[78] + 1)**2 + (-u[57] + u[79] + 1)**2)
        + 2 * (np.sqrt((-u[77] + u[79])**2 + (-u[76] + u[78] + 1)**2) - 1) * (-u[76] + u[78] + 1) / np.sqrt((-u[77] + u[79])**2 + (-u[76] + u[78] + 1)**2)
    )

    ans[79] = (
        -g[79]
        + 2 * (-u[77] + u[79]) * (np.sqrt((-u[77] + u[79])**2 + (-u[76] + u[78] + 1)**2) - 1) / np.sqrt((-u[77] + u[79])**2 + (-u[76] + u[78] + 1)**2)
        + 2 * (np.sqrt((-u[56] + u[78] + 1)**2 + (-u[57] + u[79] + 1)**2) - np.sqrt(2)) * (-u[57] + u[79] + 1) / np.sqrt((-u[56] + u[78] + 1)**2 + (-u[57] + u[79] + 1)**2)
        + 2 * (np.sqrt((-u[58] + u[78])**2 + (-u[59] + u[79] + 1)**2) - 1) * (-u[59] + u[79] + 1) / np.sqrt((-u[58] + u[78])**2 + (-u[59] + u[79] + 1)**2)
    )
    return np.array([round(i, 4) for i in ans])

In [4]:
g = np.zeros(80)

g[61] = 1
g[78] = 1

u = np.array([0.8147, 0.9058, 0.1270, 0.9134, 0.6324, 0.0975, 0.2785, 0.5469, 0.9575, 0.9649, 0.1576, 0.9706, 0.9572, 0.4854, 0.8003,
    0.1419, 0.4218, 0.9157, 0.7922, 0.9595, 0.6557, 0.0357, 0.8491, 0.9340, 0.6787, 0.7577, 0.7431, 0.3922, 0.6555, 0.1712, 0.7060,
    0.0318, 0.2769, 0.0462, 0.0971, 0.8235, 0.6948, 0.3171, 0.9502, 0.0344, 0.4387,0.3816, 0.7655, 0.7952, 0.1869, 0.4898, 0.4456,
    0.6463, 0.7094, 0.7547, 0.2760, 0.6797, 0.6551, 0.1626, 0.1190, 0.4984, 0.9597, 0.3404, 0.5853, 0.2238, 0.7513, 0.2551, 0.5060,
    0.6991, 0.8909, 0.9593, 0.5472, 0.1386, 0.1493, 0.2575, 0.8407, 0.2543, 0.8143, 0.2435, 0.9293, 0.3500, 0.1966, 0.2511, 0.6160,
    0.4733
])

grad(g, u)

array([ 8.2760e-01,  3.9456e+00, -4.9622e+00,  4.8403e+00,  2.9894e+00,
       -2.6788e+00, -1.2564e+00,  2.0678e+00,  5.9835e+00,  6.4761e+00,
       -2.9499e+00,  5.9816e+00,  4.0644e+00,  2.5033e+00,  2.7696e+00,
       -9.9080e-01,  1.1392e+00,  5.1424e+00,  4.2488e+00,  4.7884e+00,
        1.6421e+00, -3.9092e+00,  1.8731e+00,  3.6070e+00,  3.1750e-01,
        2.2978e+00,  1.5792e+00, -5.6970e-01,  3.0317e+00, -2.8986e+00,
       -2.9330e-01, -2.4925e+00, -1.6520e+00, -2.2489e+00, -4.5222e+00,
        3.6004e+00,  1.7816e+00, -2.9397e+00, -2.7970e-01, -1.6397e+00,
       -1.4639e+00,  2.2380e-01,  1.4346e+00,  5.4760e-01, -3.5096e+00,
       -1.6091e+00, -1.3810e-01,  2.3205e+00,  7.3890e-01,  3.1101e+00,
       -2.8609e+00,  3.3196e+00,  2.5639e+00, -9.8660e-01, -3.5648e+00,
        1.1061e+00,  3.5476e+00,  3.2160e-01, -5.6780e-01, -4.7660e-01,
        7.1330e-01, -1.2582e+00, -6.0560e-01, -6.0000e-04,  1.5608e+00,
        1.6810e+00,  1.1494e+00, -9.4320e-01, -2.2373e+00, -1.24

In [8]:
import numpy as np

def gradient_descent_model4a(
    g,
    u_init=None,
    alpha=1e-3,
    tol=1e-8,
    max_iter=20000
):
    """
    Basic Gradient Descent for model_4a with a fixed step size.

    Parameters:
        g : np.array
            Parameter vector (same dimension as u)
        u_init : np.array
            Initial guess (default: zeros)
        alpha : float
            Fixed step size (learning rate)
        tol : float
            Gradient norm tolerance for stopping
        max_iter : int
            Maximum number of iterations

    Returns:
        u_star : np.array
            Minimizer
        m_star : float
            Minimum value of model_4a
        iters : int
            Number of iterations
        history : list
            List of (u, m)
    """
    n = len(g)
    if u_init is None:
        u_init = np.zeros(n)

    u = np.array(u_init, dtype=float)
    m = model_4a(u, g)
    history = [(u.copy(), m)]

    for i in range(max_iter):
        grad_u = grad(u, g)
        grad_norm = np.linalg.norm(grad_u)
        if grad_norm < tol:
            break

        # update step
        u = u - alpha * grad_u
        m = model_4a(u, g)
        history.append((u.copy(), m))

    return u, m, i + 1, history


In [9]:
def gradient_descent_armijo_model4a(
    g,
    u_init=None,
    alpha_init=1.0,
    c=1e-4,
    r=0.5,
    tol=1e-8,
    max_iter=20000
):
    """
    Gradient Descent with Backtracking Line Search (Armijo rule)
    for model_4a.

    Parameters:
        g : np.array
            Parameter vector
        u_init : np.array
            Initial guess (default: zeros)
        alpha_init : float
            Initial step size for backtracking
        c : float
            Armijo parameter (0 < c < 1)
        r : float
            Step reduction factor (0 < r < 1)
        tol : float
            Gradient norm tolerance
        max_iter : int
            Maximum iterations

    Returns:
        u_star : np.array
            Minimizer
        m_star : float
            Minimum value
        iters : int
            Number of iterations
        history : list
            List of (u, m, alpha)
    """
    n = len(g)
    if u_init is None:
        u_init = np.zeros(n)

    u = np.array(u_init, dtype=float)
    m = model_4a(u, g)
    history = [(u.copy(), m, alpha_init)]

    for i in range(max_iter):
        grad_u = grad(u, g)
        grad_norm = np.linalg.norm(grad_u)
        if grad_norm < tol:
            break

        h = -grad_u
        alpha = alpha_init

        # Armijo backtracking line search
        while True:
            u_trial = u + alpha * h
            m_trial = model_4a(u_trial, g)
            if m_trial <= m + c * alpha * np.dot(h, grad_u):
                break
            alpha *= r

        # update
        u = u_trial
        m = m_trial
        history.append((u.copy(), m, alpha))

    return u, m, i + 1, history


In [ ]:
np.random.seed(0)
g = np.random.randn(80)
u0 = np.zeros(80)

# --- 1. Fixed Step Gradient Descent ---
u_fixed, m_fixed, it_fixed, hist_fixed = gradient_descent_model4a(
    g=g, u_init=u0, alpha=1e-3
)
print("Fixed-step GD → m =", m_fixed, "in", it_fixed, "iterations")

# --- 2. Armijo Backtracking Gradient Descent ---
u_armijo, m_armijo, it_armijo, hist_armijo = gradient_descent_armijo_model4a(
    g=g, u_init=u0, alpha_init=1.0, c=1e-4, r=0.5
)
print("Armijo GD → m =", m_armijo, "in", it_armijo, "iterations")

Fixed-step GD → m = 1.0209315601507393e+22 in 20000 iterations
